# 1. Import & Read

In [133]:
import pandas as pd
import numpy as np
import os
import torch
from keybert import KeyBERT
from transformers import PreTrainedTokenizerFast, BartForConditionalGeneration, AutoTokenizer, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer

from langchain_core.messages import ChatMessage, HumanMessage, AIMessage, SystemMessage
from langchain_core.embeddings import Embeddings
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

from langchain_community.document_loaders import TextLoader, PyPDFLoader, PyMuPDFLoader
from langchain_community.vectorstores import FAISS
from langchain_community.vectorstores.utils import DistanceStrategy

from langchain_openai import ChatOpenAI

from langchain.text_splitter import RecursiveCharacterTextSplitter

from langserve.pydantic_v1 import BaseModel, Field
from typing import List, Union

from openai import OpenAI
import os
import warnings
warnings.filterwarnings("ignore")

pdf_dir = "file/rag_score_pdf/"

score_df = pd.read_csv("file/rag_evaluation_result.csv")

# 오류 나는 PDF 제외
score_df = score_df.drop(axis=0, index=[135, 169, 180, 181, 182, 243, 244, 245, 246, 247, 248], inplace=True)

# 2. 답변 생성

In [2]:
RAG_PROMPT_TEMPLATE = (
    """
    You always answer into Korean. You are an assistant for question-answering tasks.
    Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know.
    Use all the sentences and keep the answer concise in bullet.\nQuestion: {question} \nContext: {context} \nAnswer:
    """
)

SYS_PROMPT_TEMPLATE = (
    """
    당신은 ** 에이전트입니다. 가능한 한 도움이 될 수 있도록 사실적인 정보만을 제공하도록 설계되었습니다. 
    친절하게 응대하지만 지나치게 수다스럽지는 않습니다. 아래에 제공된 컨텍스트 정보를 바탕으로, 사전 지식 없이 질문에 답하십시오.
    """
)

## 2-1) 토큰화 X

In [55]:
# PDF 로더 함수
def load_pdf(file_path):
    loader = PyMuPDFLoader(pdf_dir + file_path)
    return loader.load()

# 텍스트 분할 함수
def split_text(pages, tokenizer):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=256,
        chunk_overlap=20,
        length_function=lambda text: len(tokenizer.encode(text)),
    )
    return text_splitter.split_documents(pages)

# Embeddings 클래스 정의
class MyEmbeddings(Embeddings):
    def __init__(self, model_name="bespin-global/klue-sroberta-base-continue-learning-by-mnr"):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return self.model.encode(texts, convert_to_tensor=True).tolist()

    def embed_query(self, text: str) -> List[float]:
        return self.embed_documents([text])[0]

# RAG 시스템 설정 함수
def setup_rag_system(llm_base_url, embedding_model_name):
    # 언어 모델 설정
    llm = ChatOpenAI(
        base_url=llm_base_url,
        api_key="lm-studio",
        model="teddylee777/EEVE-Korean-Instruct-10.8B-v1.0-gguf",
        temperature=0.1,
    )
    # 임베딩 모델 설정
    emb = MyEmbeddings(model_name=embedding_model_name)

    # RAG 체인 설정
    rag_prompt = ChatPromptTemplate.from_messages([
        ("system", "You always answer into Korean."),
        ("user", RAG_PROMPT_TEMPLATE),
    ])
    rag_chain = rag_prompt | llm | StrOutputParser()

    return llm, emb, rag_chain

# 문서 저장소 및 검색기 설정 함수
def setup_vectorstore(docs, emb):
    vectorstore = FAISS.from_documents(docs, embedding=emb, distance_strategy=DistanceStrategy.COSINE)
    return vectorstore.as_retriever(search_kwargs={'k': 3})

# 질문 요약 함수
def summarize_question(question: str, tokenizer, bart_model) -> str:
    inputs = tokenizer.encode(question, return_tensors="pt")
    summary_ids = bart_model.generate(inputs, max_length=200, num_beams=4, early_stopping=True)
    return tokenizer.decode(summary_ids[0], skip_special_tokens=True)

# 키워드 추출 함수
def extract_keywords(text: str, keyword_model, top_n: int = 5) -> List[str]:
    return [keyword[0] for keyword in keyword_model.extract_keywords(text, top_n=top_n)]

# 문서 포맷팅 함수
def format_docs(docs):
    return " ".join(doc.page_content for doc in docs)

# 질문 처리 함수
def process_question(question: str, retriever, tokenizer, bart_model, keyword_model, chain):
    summarized_question = summarize_question(question, tokenizer, bart_model)
    keywords = extract_keywords(question, keyword_model)
    retriever_result = retriever.get_relevant_documents(" ".join(keywords))
    context = format_docs(retriever_result)

    input_data = {
        "context": context,
        "question": summarized_question
    }

    return chain.invoke(input_data)

# 전체 프로세스 실행 함수
def run_pipeline(file_path, question, llm_base_url="http://localhost:1234/v1"):
    # PDF 로드
    pages = load_pdf(file_path)

    # KoBART 모델 및 토크나이저 로드
    tokenizer = PreTrainedTokenizerFast.from_pretrained("EbanLee/kobart-summary-v3")
    bart_model = BartForConditionalGeneration.from_pretrained("EbanLee/kobart-summary-v3")

    # 텍스트 분할
    docs = split_text(pages, tokenizer)

    # RAG 시스템 및 체인 설정
    llm, emb, rag_chain = setup_rag_system(llm_base_url, "bespin-global/klue-sroberta-base-continue-learning-by-mnr")

    # 문서 저장소 및 검색기 설정
    retriever = setup_vectorstore(docs, emb)

    # KeyBERT 모델 로드
    keyword_model = KeyBERT()

    # 질문 처리 및 답변 생성
    return process_question(question, retriever, tokenizer, bart_model, keyword_model, rag_chain)

### 메모리 저장 X

In [45]:
import gc

# PDF 로더 함수
def load_pdf(file_path):
    loader = PyMuPDFLoader(pdf_dir + file_path)
    return loader.load()

# 텍스트 분할 함수
def split_text(pages, tokenizer):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=256,
        chunk_overlap=20,
        length_function=lambda text: len(tokenizer.encode(text)),
    )
    split_docs = text_splitter.split_documents(pages)

    # 메모리에서 불필요한 데이터 제거
    del pages
    gc.collect()

    return split_docs

# Embeddings 클래스 정의
class MyEmbeddings(Embeddings):
    def __init__(self, model_name="bespin-global/klue-sroberta-base-continue-learning-by-mnr"):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        embeddings = self.model.encode(texts, convert_to_tensor=True).tolist()

        # 메모리에서 불필요한 데이터 제거
        del texts
        gc.collect()

        return embeddings

    def embed_query(self, text: str) -> List[float]:
        return self.embed_documents([text])[0]

# RAG 시스템 설정 함수
def setup_rag_system(llm_base_url, embedding_model_name):
    # 언어 모델 설정
    llm = ChatOpenAI(
        base_url=llm_base_url,
        api_key="lm-studio",
        model="teddylee777/EEVE-Korean-Instruct-10.8B-v1.0-gguf",
        temperature=0.1,
    )
    # 임베딩 모델 설정
    emb = MyEmbeddings(model_name=embedding_model_name)

    # RAG 체인 설정
    rag_prompt = ChatPromptTemplate.from_messages([
        ("system", "You always answer into Korean."),
        ("user", RAG_PROMPT_TEMPLATE),
    ])
    rag_chain = rag_prompt | llm | StrOutputParser()

    return llm, emb, rag_chain

# 문서 저장소 및 검색기 설정 함수
def setup_vectorstore(docs, emb):
    vectorstore = FAISS.from_documents(docs, embedding=emb, distance_strategy=DistanceStrategy.COSINE)

    # 메모리에서 불필요한 데이터 제거
    del docs
    gc.collect()

    return vectorstore.as_retriever(search_kwargs={'k': 3})

# 질문 요약 함수
def summarize_question(question: str, tokenizer, bart_model) -> str:
    inputs = tokenizer.encode(question, return_tensors="pt")
    summary_ids = bart_model.generate(inputs, max_length=200, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # 메모리에서 불필요한 데이터 제거
    del inputs, summary_ids
    gc.collect()

    return summary

# 키워드 추출 함수
def extract_keywords(text: str, keyword_model, top_n: int = 5) -> List[str]:
    keywords = [keyword[0] for keyword in keyword_model.extract_keywords(text, top_n=top_n)]

    # 메모리에서 불필요한 데이터 제거
    gc.collect()

    return keywords

# 문서 포맷팅 함수
def format_docs(docs):
    formatted_docs = " ".join(doc.page_content for doc in docs)

    # 메모리에서 불필요한 데이터 제거
    del docs
    gc.collect()

    return formatted_docs

# 질문 처리 함수
def process_question(question: str, retriever, tokenizer, bart_model, keyword_model, chain):
    summarized_question = summarize_question(question, tokenizer, bart_model)
    keywords = extract_keywords(question, keyword_model)
    retriever_result = retriever.get_relevant_documents(" ".join(keywords))
    context = format_docs(retriever_result)

    input_data = {
        "context": context,
        "question": summarized_question
    }

    # 메모리에서 불필요한 데이터 제거
    del summarized_question, keywords, retriever_result, context
    gc.collect()

    return chain.invoke(input_data)

# 전체 프로세스 실행 함수
def run_pipeline(file_path, question, llm_base_url="http://localhost:1234/v1"):
    # PDF 로드
    pages = load_pdf(file_path)

    # KoBART 모델 및 토크나이저 로드
    tokenizer = PreTrainedTokenizerFast.from_pretrained("EbanLee/kobart-summary-v3")
    bart_model = BartForConditionalGeneration.from_pretrained("EbanLee/kobart-summary-v3")

    # 텍스트 분할
    docs = split_text(pages, tokenizer)

    # RAG 시스템 및 체인 설정
    llm, emb, rag_chain = setup_rag_system(llm_base_url, "bespin-global/klue-sroberta-base-continue-learning-by-mnr")

    # 문서 저장소 및 검색기 설정
    retriever = setup_vectorstore(docs, emb)

    # KeyBERT 모델 로드
    keyword_model = KeyBERT()

    # 질문 처리 및 답변 생성
    result = process_question(question, retriever, tokenizer, bart_model, keyword_model, rag_chain)

    # 메모리에서 불필요한 데이터 제거
    del docs, retriever, tokenizer, bart_model, keyword_model, rag_chain, emb, llm
    gc.collect()

    return result

## 2-2) 토큰화 O

In [ ]:
# PDF 로더 함수
def load_pdf(file_path):
    loader = PyMuPDFLoader(pdf_dir + file_path)
    pages = loader.load()
    return pages

# 텍스트 분할 함수
def split_text(pages, tokenizer):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=256,
        chunk_overlap=20,
        separators=["\n\n", "\n", "(?<=\. )", " ", ""],
        length_function=lambda text: len(tokenizer.encode(text)),
    )
    docs = text_splitter.split_documents(pages)
    return docs

# 텍스트 클리닝 함수
def clean_text(text):
    return ' '.join(text.split())

# 문서 토크나이징 함수
def tokenize_docs(docs, tokenizer):
    tokenized_docs = []
    for doc in docs:
        tokenized_text = tokenizer(clean_text(doc.page_content), return_tensors="pt")
        tokenized_docs.append(tokenized_text)
    return tokenized_docs

# Embeddings 클래스 정의
class MyEmbeddings(Embeddings):
    def __init__(self, model_name="bespin-global/klue-sroberta-base-continue-learning-by-mnr"):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        embeddings = self.model.encode(texts, convert_to_tensor=True)
        return embeddings.tolist()

    def embed_query(self, text: str) -> List[float]:
        embedding = self.model.encode([text], convert_to_tensor=True)
        return embedding[0].tolist()

# RAG 시스템 설정 함수
def setup_rag_system(llm_base_url, embedding_model_name="bespin-global/klue-sroberta-base-continue-learning-by-mnr"):
    # 언어 모델 설정
    llm = ChatOpenAI(
        base_url=llm_base_url,
        api_key="lm-studio",
        model="teddylee777/EEVE-Korean-Instruct-10.8B-v1.0-gguf",
        temperature=0.1,
    )

    # 임베딩 모델 설정
    emb = MyEmbeddings(model_name=embedding_model_name)

    # RAG 체인 설정
    rag_prompt = ChatPromptTemplate.from_messages([
        ("system", "You always answer into Korean."),
        ("user", "Context: {context}\n\nQuestion: {question}\n\nAnswer:"),
    ])
    rag_chain = rag_prompt | llm | StrOutputParser()

    return llm, emb, rag_chain

# 문서 저장소 및 검색기 설정 함수
def setup_vectorstore(docs, emb):
    vectorstore = FAISS.from_documents(docs, embedding=emb, distance_strategy=DistanceStrategy.COSINE)
    retriever = vectorstore.as_retriever(search_kwargs={'k': 3})
    return vectorstore, retriever

# 질문 요약 함수
def summarize_question(question: str, tokenizer, bart_model) -> str:
    inputs = tokenizer.encode(question, return_tensors="pt")
    summary_ids = bart_model.generate(inputs, max_length=200, min_length=0, length_penalty=2.0, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# 키워드 추출 함수
def extract_keywords(text: str, keyword_model, top_n: int = 5) -> List[str]:
    keywords = keyword_model.extract_keywords(text, top_n=top_n)
    return [keyword[0] for keyword in keywords]

# 문서 포맷팅 함수
def format_docs(docs):
    return " ".join(doc.page_content for doc in docs)

# 질문 처리 함수
def process_question(question: str, retriever, tokenizer, bart_model, keyword_model, chain, docs):
    summarized_question = summarize_question(question, tokenizer, bart_model)
    keywords = extract_keywords(question, keyword_model)
    retriever_result = retriever.get_relevant_documents(" ".join(keywords))
    context = format_docs(retriever_result)

    # 토크나이징된 문서 활용
    tokenized_contexts = tokenize_docs(docs, tokenizer)
    # 토크나이징된 문서로 context 업데이트
    # 예시: 단순히 첫 번째 토크나이징된 문서를 사용
    if tokenized_contexts:
        context += " " + tokenizer.decode(tokenized_contexts[0]['input_ids'][0], skip_special_tokens=True)

    input_data = {
        "context": context,
        "question": summarized_question
    }

    answer = chain.invoke(input_data)
    return answer

# 전체 프로세스 실행 함수
def run_pipeline(file_path, question, llm_base_url="http://localhost:1234/v1"):
    # PDF 로드
    pages = load_pdf(file_path)

    # KoBART 모델 및 토크나이저 로드
    tokenizer = PreTrainedTokenizerFast.from_pretrained("EbanLee/kobart-summary-v3")
    bart_model = BartForConditionalGeneration.from_pretrained("EbanLee/kobart-summary-v3")

    # 텍스트 분할
    docs = split_text(pages, tokenizer)

    # RAG 시스템 및 체인 설정
    llm, emb, rag_chain = setup_rag_system(llm_base_url)

    # 문서 저장소 및 검색기 설정
    vectorstore, retriever = setup_vectorstore(docs, emb)

    # KeyBERT 모델 로드
    keyword_model = KeyBERT()

    # 질문 처리 및 답변 생성
    answer = process_question(question, retriever, tokenizer, bart_model, keyword_model, rag_chain, docs)

    return answer

## 번외

### 1) 질문 요약 X

In [ ]:
import gc

# PDF 로더 함수
def load_pdf(file_path):
    loader = PyMuPDFLoader(pdf_dir + file_path)
    return loader.load()

# 텍스트 분할 함수
def split_text(pages, tokenizer):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=256,
        chunk_overlap=20,
        length_function=lambda text: len(tokenizer.encode(text)),
    )
    split_docs = text_splitter.split_documents(pages)

    # 메모리에서 불필요한 데이터 제거
    del pages
    gc.collect()

    return split_docs

# Embeddings 클래스 정의
class MyEmbeddings(Embeddings):
    def __init__(self, model_name="bespin-global/klue-sroberta-base-continue-learning-by-mnr"):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        embeddings = self.model.encode(texts, convert_to_tensor=True).tolist()

        # 메모리에서 불필요한 데이터 제거
        del texts
        gc.collect()

        return embeddings

    def embed_query(self, text: str) -> List[float]:
        return self.embed_documents([text])[0]

# RAG 시스템 설정 함수
def setup_rag_system(llm_base_url, embedding_model_name):
    # 언어 모델 설정
    llm = ChatOpenAI(
        base_url=llm_base_url,
        api_key="lm-studio",
        model="teddylee777/EEVE-Korean-Instruct-10.8B-v1.0-gguf",
        temperature=0.1,
    )
    # 임베딩 모델 설정
    emb = MyEmbeddings(model_name=embedding_model_name)

    # RAG 체인 설정
    rag_prompt = ChatPromptTemplate.from_messages([
        ("system", "You always answer into Korean."),
        ("user", RAG_PROMPT_TEMPLATE),
    ])
    rag_chain = rag_prompt | llm | StrOutputParser()

    return llm, emb, rag_chain

# 문서 저장소 및 검색기 설정 함수
def setup_vectorstore(docs, emb):
    vectorstore = FAISS.from_documents(docs, embedding=emb, distance_strategy=DistanceStrategy.COSINE)

    # 메모리에서 불필요한 데이터 제거
    del docs
    gc.collect()

    return vectorstore.as_retriever(search_kwargs={'k': 3})

# 키워드 추출 함수
def extract_keywords(text: str, keyword_model, top_n: int = 5) -> List[str]:
    keywords = [keyword[0] for keyword in keyword_model.extract_keywords(text, top_n=top_n)]

    # 메모리에서 불필요한 데이터 제거
    gc.collect()

    return keywords

# 문서 포맷팅 함수
def format_docs(docs):
    formatted_docs = " ".join(doc.page_content for doc in docs)

    # 메모리에서 불필요한 데이터 제거
    del docs
    gc.collect()

    return formatted_docs

# 질문 처리 함수
def process_question(question: str, retriever, tokenizer, bart_model, keyword_model, chain):
    keywords = extract_keywords(question, keyword_model)
    retriever_result = retriever.get_relevant_documents(" ".join(keywords))
    context = format_docs(retriever_result)

    input_data = {
        "context": context,
        "question": question
    }

    # 메모리에서 불필요한 데이터 제거
    del keywords, retriever_result, context
    gc.collect()

    return chain.invoke(input_data)

# 전체 프로세스 실행 함수
def run_pipeline(file_path, question, llm_base_url="http://localhost:1234/v1"):
    # PDF 로드
    pages = load_pdf(file_path)

    # KoBART 모델 및 토크나이저 로드
    tokenizer = PreTrainedTokenizerFast.from_pretrained("EbanLee/kobart-summary-v3")
    bart_model = BartForConditionalGeneration.from_pretrained("EbanLee/kobart-summary-v3")

    # 텍스트 분할
    docs = split_text(pages, tokenizer)

    # RAG 시스템 및 체인 설정
    llm, emb, rag_chain = setup_rag_system(llm_base_url, "bespin-global/klue-sroberta-base-continue-learning-by-mnr")

    # 문서 저장소 및 검색기 설정
    retriever = setup_vectorstore(docs, emb)

    # KeyBERT 모델 로드
    keyword_model = KeyBERT()

    # 질문 처리 및 답변 생성
    result = process_question(question, retriever, tokenizer, bart_model, keyword_model, rag_chain)

    # 메모리에서 불필요한 데이터 제거
    del docs, retriever, tokenizer, bart_model, keyword_model, rag_chain, emb, llm
    gc.collect()

    return result

### 2) ReRanking / 질문 요약 O

In [3]:
import gc

# PDF 로더 함수
def load_pdf(file_path):
    loader = PyMuPDFLoader(pdf_dir + file_path)
    return loader.load()

# 텍스트 분할 함수
def split_text(pages, tokenizer):
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=256,
        chunk_overlap=20,
        separators=["\n\n", "\n", "(?<=\. )", " ", ""],
        length_function=lambda text: len(tokenizer.encode(text)),
    )
    split_docs = text_splitter.split_documents(pages)

    # 메모리에서 불필요한 데이터 제거
    del pages
    gc.collect()

    return split_docs

# Embeddings 클래스 정의
class MyEmbeddings(Embeddings):
    def __init__(self, model_name="bespin-global/klue-sroberta-base-continue-learning-by-mnr"):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        embeddings = self.model.encode(texts, convert_to_tensor=True).tolist()

        # 메모리에서 불필요한 데이터 제거
        del texts
        gc.collect()

        return embeddings

    def embed_query(self, text: str) -> List[float]:
        return self.embed_documents([text])[0]

# RAG 시스템 설정 함수
def setup_rag_system(llm_base_url, embedding_model_name):
    # 언어 모델 설정
    llm = ChatOpenAI(
        base_url=llm_base_url,
        api_key="lm-studio",
        model="teddylee777/EEVE-Korean-Instruct-10.8B-v1.0-gguf",
        temperature=0.1,
    )
    # 임베딩 모델 설정
    emb = MyEmbeddings(model_name=embedding_model_name)

    # RAG 체인 설정
    rag_prompt = ChatPromptTemplate.from_messages([
        ("system", "You always answer into Korean."),
        ("user", RAG_PROMPT_TEMPLATE),
    ])
    rag_chain = rag_prompt | llm | StrOutputParser()

    return llm, emb, rag_chain

# 문서 저장소 및 검색기 설정 함수
def setup_vectorstore(docs, emb):
    vectorstore = FAISS.from_documents(docs, embedding=emb, distance_strategy=DistanceStrategy.COSINE)

    # 메모리에서 불필요한 데이터 제거
    del docs
    gc.collect()

    return vectorstore.as_retriever(search_kwargs={'k': 5})

# 질문 요약 함수
def summarize_question(question: str, tokenizer, bart_model) -> str:
    inputs = tokenizer.encode(question, return_tensors="pt")
    summary_ids = bart_model.generate(inputs, max_length=256, num_beams=4, early_stopping=True)
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

    # 메모리에서 불필요한 데이터 제거
    del inputs, summary_ids
    gc.collect()

    return summary


# 키워드 추출 함수
def extract_keywords(text: str, keyword_model, top_n: int = 5) -> List[str]:
    keywords = [keyword[0] for keyword in keyword_model.extract_keywords(text, top_n=top_n)]

    # 메모리에서 불필요한 데이터 제거
    gc.collect()

    return keywords

# 문서 포맷팅 함수
def format_docs(docs):
    formatted_docs = " ".join(doc.page_content for doc in docs)

    # 메모리에서 불필요한 데이터 제거
    del docs
    gc.collect()

    return formatted_docs

# 질문 처리 함수
def process_question(question: str, retriever, tokenizer, bart_model, keyword_model, chain):
    model_path = "Dongjin-kr/ko-reranker"

    rerank_tokenizer = AutoTokenizer.from_pretrained(model_path)
    rerank_model = AutoModelForSequenceClassification.from_pretrained(model_path)
    rerank_model.eval()

    summarized_question = summarize_question(question, tokenizer, bart_model)
    keywords = extract_keywords(question, keyword_model)
    retriever_result = retriever.get_relevant_documents(" ".join(keywords))

    pairs = []

    for rs in retriever_result:
        pairs.append([question, rs.page_content])

    # Reranking 수행
    with torch.no_grad():
        inputs = rerank_tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
        scores = rerank_model(**inputs, return_dict=True).logits.view(-1, ).float()
        scores = normalize(scores.numpy())

    # 상위 3개의 점수 인덱스 가져오기
    top_3_indices = scores.argsort()[-3:][::-1]  # 점수가 높은 순서대로 상위 3개의 인덱스

    # 상위 3개의 문서 추출
    top_3_docs = [retriever_result[i] for i in top_3_indices]

    context = format_docs(top_3_docs)

    input_data = {
        "context": context,
        "question": summarized_question
    }

    # 메모리에서 불필요한 데이터 제거
    del summarized_question, keywords, retriever_result, context, top_3_indices, top_3_docs, pairs
    gc.collect()

    return chain.invoke(input_data)

def normalize(x):
    b = x.max()
    y = np.exp(x - b)
    return y / y.sum()


# 전체 프로세스 실행 함수
def run_pipeline(file_path, question, llm_base_url="http://localhost:1234/v1"):
    # PDF 로드
    pages = load_pdf(file_path)

    # KoBART 모델 및 토크나이저 로드
    tokenizer = PreTrainedTokenizerFast.from_pretrained("EbanLee/kobart-summary-v3")
    bart_model = BartForConditionalGeneration.from_pretrained("EbanLee/kobart-summary-v3")

    # 텍스트 분할
    docs = split_text(pages, tokenizer)

    # RAG 시스템 및 체인 설정
    llm, emb, rag_chain = setup_rag_system(llm_base_url, "bespin-global/klue-sroberta-base-continue-learning-by-mnr")

    # 문서 저장소 및 검색기 설정
    retriever = setup_vectorstore(docs, emb)

    # KeyBERT 모델 로드
    keyword_model = KeyBERT()

    # 질문 처리 및 답변 생성
    result = process_question(question, retriever, tokenizer, bart_model, keyword_model, rag_chain)

    # 메모리에서 불필요한 데이터 제거
    del docs, retriever, tokenizer, bart_model, keyword_model, rag_chain, emb, llm
    gc.collect()

    return result

# 3. 답변 생성

In [24]:
answers = []

for index, row in score_df.iterrows():
    # 각 행의 파일명과 질문을 사용하여 run_pipeline 실행
    file_path = row['target_file_name']
    question = row['question']
    
    # run_pipeline 함수 호출
    answers.append(run_pipeline(file_path=file_path, question=question))

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', 

IndexError: list index out of range

In [25]:
my_model = pd.DataFrame(answers)
my_model.columns = ['answer']
my_model.to_csv("file/my_model.csv")

In [28]:
len(answers)

135

In [5]:
score_df_test = score_df.iloc[136:]
score_df_test

,domain,question,target_answer,target_file_name,target_page_no,context_type,alli_gpt-4-turbo_answer,alli_gpt-4-turbo_ox,alli_gpt-4_answer,alli_gpt-4_ox,...,anything_llm_claude3-opus_answer,anything_llm_claude3-opus_ox,upstage_gpt-3.5-turbo_answer,upstage_gpt-3.5-turbo_ox,upstage_gpt-4-turbo_answer,upstage_gpt-4-turbo_ox,upstage_gpt-4_answer,upstage_gpt-4_ox,upstage_solar_answer,upstage_solar_ox
136,medical,"아데노바이러스 중 어떤 형이 각결막염의 중증 사례의 주원인이며, 이것이 전세계적으로...","아데노바이러스 8형, 5형, 19형이 각결막염의 중증 사례의 주 원인이며, 이 중 ...",2017년-안과감염병-관리지침.pdf,16,text,"아데노바이러스 중 각결막염의 중증 사례의 주 원인은 아데노바이러스 8형, 5형, 1...",O,"각결막염의 중증 사례의 주 원인은 아데노바이러스 8형, 5형, 19형입니다[1]. ...",O,...,"아데노바이러스 중 각결막염의 중증 사례를 주로 일으키는 형은 아데노바이러스 8형, ...",O,"아데노바이러스 8형이 중증 각결막염의 주요 원인이며, 전세계적으로 주로 보고된 혈청...",O,"아데노바이러스 중 각결막염의 중증 사례의 주원인은 아데노바이러스 8형, 19형, 3...",O,"각결막염의 중증 사례의 주원인은 아데노바이러스 8형, 5형, 19형이다. 전세계적으...",O,"아데노바이러스 중 각결막염의 중증 사례의 주원인은 8형, 5형, 19형이며, 전세계...",O
137,medical,2014년~2016년 동안 유행성 각결막염의 환자 분율이 가장 높았던 주와 그 때의...,"2014년도 37주에 환자 분율이 가장 높았으며, 그 때의 환자 분율은 28.5명이...",2017년-안과감염병-관리지침.pdf,17,text,2014년부터 2016년까지 유행성 각결막염의 환자 분율이 가장 높았던 주와 그때의...,O,"2014년에는 37주에 환자 분율이 28.5명, 2015년에는 40주에 환자 분율이...",O,...,2014년부터 2016년까지 유행성 각결막염 환자 분율이 가장 높았던 주(week)...,X,2014년~2016년 동안 유행성 각결막염의 환자 분율이 가장 높았던 주는 2016...,O,"2014년에 환자 분율이 가장 높았던 주는 37주로, 환자 분율은 28.5명이었습니...",O,"2014년에는 37주에 환자 분율이 28.5명으로 가장 높았고, 2015년에는 40...",O,2014년~2016년 동안 유행성 각결막염의 환자 분율이 가장 높았던 주는 2014...,O
138,medical,급성 여포성 결막염의 합병증 중 안구 건조증이 발생하는 원인과 그보통 지속되는 기간...,"급성 여포성 결막염의 합병증 중 하나인 안구 건조증은, 결막에 분비된 눈물보조세포나...",2017년-안과감염병-관리지침.pdf,19,text,급성 여포성 결막염의 합병증 중 하나로 안구 건조증이 발생할 수 있습니다. 이는 결...,O,급성 여포성 결막염의 합병증 중 하나인 안구 건조증은 결막에 분비된 눈물보조세포나 ...,O,...,급성 여포성 결막염의 합병증 중 하나인 안구 건조증은 결막염으로 인한 결막 표면의 ...,O,"안구 건조증은 급성 여포성 결막염의 합병증 중 하나로, 결막에 분비된 눈물보조세포나...",O,급성 여포성 결막염에서 안구 건조증은 결막에 있는 눈물보조세포나 점액분비 세포가 바...,O,급성 여포성 결막염의 합병증 중 안구 건조증은 결막에 분비된 눈물보조세포나 점액분비...,O,급성 여포성 결막염의 합병증 중 안구 건조증이 발생하는 원인은 결막에 분비된 눈물보...,O
139,medical,각막 상피하 혼탁 발생 시 쓰이는 약물과 이차적인 세균감염을 예방하는 데 사용되는 ...,"각막 상피하 혼탁이 발생한 경우 점안 스테로이드제를 사용하며, 이차적인 세균감염을 ...",2017년-안과감염병-관리지침.pdf,20,text,"각막 상피하 혼탁이 발생한 경우 점안 스테로이드제를 사용하고, 이차적인 세균감염을 ...",O,"각막 상피하 혼탁이 발생한 경우에는 점안 스테로이드제를 사용하며, 이차적인 세균감염...",O,...,각막 상피하 혼탁 발생 시 사용되는 약물과 이차적인 세균감염 예방을 위해 사용되는 ...,O,"각막 상피하 혼탁 발생 시에는 점안 스테로이드제를 사용하고, 이차적인 세균감염을 예...",O,각막 상피하 혼탁이 발생했을 때 사용되는 약물은 점안 스테로이드제입니다. 이차적인 ...,O,각막 상피하 혼탁이 발생한 경우 점안 스테로이드제가 사용됩니다. 이차적인 세균감염을...,O,각막 상피하 혼탁 발생 시 점안 스테로이드제를 사용합니다. 이차적인 세균감염을 예방...,O
140,medical,"아폴로 눈병'이라고 불리는 급성 출혈성 결막염의 원인이 되는 바이러스는 무엇이며, ...","아폴로 눈병'의 원인은 EV70입니다. 이후 1986년에는 아메리칸 사모아, 199...",2017년-안과감염병-관리지침.pdf,24,text,아폴로 눈병'이라고 불리는 급성 출혈성 결막염의 원인 바이러스는 엔테로바이러스 70...,O,아폴로 눈병'이라고 불리는 급성 출혈성 결막염의 원인은 엔테로바이러스 70(EV70...,O,...,아폴로 눈병'이라고 불리는 급성 출혈성 결막염의 주요 원인 바이러스는 엔테로바이러스...,O,아폴로 눈병'의 원인 바이러스는 엔테로바이러스 70(EV70)형과 콕사키바이러스 A...,O,"급성 출혈성 결막염, 일명 '아폴로 눈병'의 원인 바이러스는 엔테로바이러스 70(E...",O,아폴로 눈병'이라고 불리는 급성 출혈성 결막염의 원인이 되는 바이러스는 엔테로바이러...,O,아폴로 눈병이라고 불리는 급성 출혈성 결막염의 원인은 엔테로바이러스 70(EV70)...,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,commerce,2020년 기업의 자체 라방 거래액이 전체 생방송 라이브커머스 대비 차지하는 비율과...,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,중국21-18호 2021 라이브커머스 산업 보고.pdf,12,paragraph,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,O,2020년에 기업의 자체 라이브 방송 거래액은 전체 생방송 라이브커머스 대비 32....,O,...,2020년 기업의 자체 라이브 방송 거래액이 전체 라이브커머스에서 차지하는 비중은 ...,O,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,O,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,O,2020년에 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 ...,O,2020년 기업의 자체 라방 거래액이 전체 생방송 라이브커머스 대비 차지하는 비율은...,O
296,commerce,"콰이서우와 틱톡에서 거래비율이 급성장한 연도와 그 이유, 그리고 현재 거래액은 어떻...",2020년 콰이서우와 틱톡의 거래비율이 급성장 했습니다. 이는 두 플랫폼이 브랜드 ...,중국21-18호 2021 라이브커머스 산업 보고.pdf,16,paragraph,콰이서우와 틱톡에서 거래비율이 급성장한 연도는 2020년입니다. 이들 플랫폼은 브랜...,O,2020년에 콰이서우와 틱톡 플랫폼 내부 거래비율이 급성장하였습니다. 이는 두 플랫...,O,...,콰이쇼우(Kuaishou)와 틱톡(Douyin)은 중국의 대표적인 숏클립 플랫폼으로...,O,"2020년에는 콰이서우와 틱톡의 플랫폼 내부 거래비율이 급성장하여, 현재 거래액의 ...",O,"2020년에 콰이서우와 틱톡 플랫폼 내부의 거래

In [6]:
answers2 = ['Error']

for index, row in score_df_test.iterrows():
    # 각 행의 파일명과 질문을 사용하여 run_pipeline 실행
    file_path = row['target_file_name']
    question = row['question']
    
    # run_pipeline 함수 호출
    answers2.append(run_pipeline(file_path=file_path, question=question))

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map

IndexError: list index out of range

In [8]:
my_model_2 = pd.DataFrame(answers2)
my_model_2.columns = ['answer']
my_model_2.to_csv("file/my_model_2.csv")

In [10]:
len(answers2)

34

In [15]:
score_df_test = score_df.iloc[136 + 34:]
score_df_test

,domain,question,target_answer,target_file_name,target_page_no,context_type,alli_gpt-4-turbo_answer,alli_gpt-4-turbo_ox,alli_gpt-4_answer,alli_gpt-4_ox,...,anything_llm_claude3-opus_answer,anything_llm_claude3-opus_ox,upstage_gpt-3.5-turbo_answer,upstage_gpt-3.5-turbo_ox,upstage_gpt-4-turbo_answer,upstage_gpt-4-turbo_ox,upstage_gpt-4_answer,upstage_gpt-4_ox,upstage_solar_answer,upstage_solar_ox
170,medical,2001년에 조사된 위암 환자들 중 증상이 있는 경우와 없는 경우의 비율 차이는 어...,증상이 있는 경우는 36%였고 증상이 없는 경우는 78%였습니다.,4e304ca4ed0e0b678457e0cf26e372f42395.pdf,2,text,"2001년에 조사된 위암 환자들 중 증상이 있는 경우와 없는 경우의 비율 차이는, ...",O,"2001년에 조사된 위암 환자들 중, 무증상기에 검진에 의해 발견된 위암의 경우 조...",O,...,2001년에 조사된 위암 환자들 중 증상이 있는 경우와 없는 경우의 비율은 다음과 ...,X,2001년에 조사된 위암 환자들 중 증상이 있는 경우와 없는 경우의 비율 차이는 증...,X,2001년에 조사된 위암 환자들 중 무증상기에 검진으로 발견된 경우의 조기위암 비율...,O,"2001년에 조사된 위암 환자들 중에서, 무증상기에 검진에 의하여 발견된 위암의 경...",O,2001년에 조사된 위암 환자들 중 증상이 있는 경우와 없는 경우의 비율 차이는 3...,O
171,medical,"조기 위암 스크리닝검사의 주기로 어떤 시기가 최적이라고 제안되었으며, 그 검사 주기...",2년 간격이 최적의 검사주기라고 제안되었습니다. 국립암센터의 연구에서는 2년 이내에...,4e304ca4ed0e0b678457e0cf26e372f42395.pdf,3,text,조기 위암 스크리닝 검사의 주기로는 2년이 적절할 것으로 제안되었습니다. 이러한 검...,O,위암 스크리닝 검사의 주기로는 2년이 적절할 것으로 제시되었습니다[1]. 이는 일본...,O,...,조기 위암 스크리닝검사의 최적 주기에 대해서는 여러 연구와 가이드라인에서 다양한 제...,O,"2년 간격의 내시경검사가 조기 위암을 발견하는 데 효과적이라고 제안되었으며, 2년 ...",O,일본의 연구에 따르면 조기 위암 스크리닝 검사의 최적 주기는 2년으로 제안되었습니다...,O,"일본의 연구에 따르면, 2년 간격으로 위암 스크리닝 검사를 받는 것이 적절하다고 제...",O,조기 위암 스크리닝검사의 주기로 2년이 최적이라고 제안되었습니다. 2년 이내에 내시...,O
172,medical,위암 환자에게 사용되는 치료 방법 중 하나로 환자의 삶의 질까지 고려한 치료 방법에...,조기위암에 대하여 내시경 치료 및 복강경 치료 등의 덜 침습적인 방식의 치료가 보편...,4e304ca4ed0e0b678457e0cf26e372f42395.pdf,8,text,위암 환자에게 사용되는 치료 방법 중 하나로 복강경수술이 있습니다. 이 방법은 내시...,O,위암 환자에게 사용되는 치료 방법 중 하나로 복강경수술이 있습니다. 이 방법은 내시...,O,...,위암 환자의 치료에 있어 삶의 질을 고려하는 것은 매우 중요합니다. 단순히 생존율 ...,X,"위암 환자에게 사용되는 치료 중 하나는 내시경 검사를 통한 조기 발견이며, 2년 간...",X,위암 환자에게 사용되는 삶의 질을 고려한 치료 방법 중 하나는 내시경 치료 및 복강...,O,위암 환자에게 사용되는 치료 방법 중 하나로는 내시경 치료가 있습니다. 이 치료 방...,O,위암 환자에게 사용되는 치료 방법 중 하나로 환자의 삶의 질까지 고려한 치료 방법은...,O
173,medical,EE와 NERD의 정의 및 공통점과 차이점을 설명해주세요.,"EE는 내시경적으로 원위부 식도 점막의 육안적인 손상이 있는 경우로 정의되며, NE...",위식도역류성질환(Gastroesophageal Reflux Disease) (2).pdf,1,text,EE(미란성식도염)와 NERD(비미란성역류질환)는 위식도역류성질환(GERD)의 두 ...,O,EE(미란성식도염)와 NERD(비미란성역류질환)는 모두 위식도역류질환(GERD)의 ...,O,...,EE(Erosive Esophagitis)와 NERD(Non-Erosive Refl...,O,EE와 NERD는 모두 위식도역류성질환의 서로 다른 서브타입으로 분류된다. EE는 ...,O,미란성식도염(EE)은 내시경적으로 식도 점막에 육안적인 손상이 확인되는 경우를 말하...,O,미란성식도염 (EE)는 내시경적으로 원위부 식도 점막의 육안적인 손상이 있는 경우로...,O,위식도역류질환(GERD)은 위 내용물이 식도로 역류하여 증상을 유발하거나 합병증을 ...,O
174,medical,"흉부작열감은 어떤 상황에서 주로 유발되는가, 그리고 그 증상을 완화 시키기 위한 방...","흉부작열감은 과식이나 맵고 지방이 많이 포함된 음식, 초콜릿, 술 등 특정 음식을 ...",위식도역류성질환(Gastroesophageal Reflux Disease) (2).pdf,3,text,"흉부작열감은 흉골 뒤쪽 가슴이 타는 듯한 증상으로, 과식을 하거나 맵고 지방을 많이...",O,"흉부작열감은 흉골 뒤쪽 가슴이 타는 듯한 증상으로, 과식을 하거나 맵고 지방을 많이...",O,...,"흉부작열감은 위식도역류질환(GERD)의 대표적인 증상으로, 식도 하부로 역류된 위산...",O,"흉부작열감은 과식이나 특정 음식을 섭취한 후 주로 유발되며, 눕거나 구부릴 때 악화...",O,"흉부작열감은 과식, 맵고 지방이 많은 음식, 초콜릿, 술 등을 섭취한 후에 주로 유...",O,"흉부작열감은 과식을 하거나 매운 음식, 지방이 많이 들어간 음식, 초콜릿, 술 등을...",O,"흉부작열감은 일반적으로 과식, 매운 음식, 지방이 많은 음식, 초콜릿, 술과 같은 ...",O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,commerce,2020년 기업의 자체 라방 거래액이 전체 생방송 라이브커머스 대비 차지하는 비율과...,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,중국21-18호 2021 라이브커머스 산업 보고.pdf,12,paragraph,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,O,2020년에 기업의 자체 라이브 방송 거래액은 전체 생방송 라이브커머스 대비 32....,O,...,2020년 기업의 자체 라이브 방송 거래액이 전체 라이브커머스에서 차지하는 비중은 ...,O,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,O,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,O,2020년에 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 ...,O,2020년 기업의 자체 라방 거래액이 전체 생방송 라이브커머스 대비 차지하는 비율은...,O
296,commerce,"콰이서우와 틱톡에서 거래비율이 급성장한 연도와 그 이유, 그리고 현재 거래액은 어떻...",2020년 콰이서우와 틱톡의 거래비율이 급성장 했습니다. 이는 두 플랫폼이 브랜드 ...,중국21-18호 2021 라이브커머스 산업 보고.pdf,16,paragraph,콰이서우와 틱톡에서 거래비율이 급성장한 연도는 2020년입니다. 이들 플랫폼은 브랜...,O,2020년에 콰이서우와 틱톡 플랫폼 내부 거래비율이 급성장하였습니다. 이는 두 플랫...,O,...,콰이쇼우(Kuaishou)와 틱톡(Douyin)은 중국의 대표적인 숏클립 플랫폼으로...,O,"2020년에는

In [16]:
answers3 = ['Error']

for index, row in score_df_test.iterrows():
    # 각 행의 파일명과 질문을 사용하여 run_pipeline 실행
    file_path = row['target_file_name']
    question = row['question']
    
    # run_pipeline 함수 호출
    answers3.append(run_pipeline(file_path=file_path, question=question))

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', 

IndexError: list index out of range

In [17]:
my_model_3 = pd.DataFrame(answers3)
my_model_3.columns = ['answer']
my_model_3.to_csv("file/my_model_3.csv")

In [19]:
len(answers3)

11

In [27]:
score_df_test = score_df.iloc[136 + 34 + 13:]
score_df_test

,domain,question,target_answer,target_file_name,target_page_no,context_type,alli_gpt-4-turbo_answer,alli_gpt-4-turbo_ox,alli_gpt-4_answer,alli_gpt-4_ox,...,anything_llm_claude3-opus_answer,anything_llm_claude3-opus_ox,upstage_gpt-3.5-turbo_answer,upstage_gpt-3.5-turbo_ox,upstage_gpt-4-turbo_answer,upstage_gpt-4-turbo_ox,upstage_gpt-4_answer,upstage_gpt-4_ox,upstage_solar_answer,upstage_solar_ox
183,law,"A가 개발한 이 사건 성능조절기능이 탑재된 J 버전 업데이트는 언제 배포되었고, 이...",피고 A가 개발한 이 사건 성능조절기능이 탑재된 J 버전 업데이트는 2017년 1월...,[민사] 스마트폰 성능조절기능이 탑재된 소프트웨어 업데이트로 인한 일부 성능제한 또...,4,paragraph,A가 개발한 이 사건 성능조절기능이 탑재된 J 버전 업데이트는 2017년 1월 23...,O,A는 2017년 1월 23일에 이 사건 성능조절기능이 탑재된 J 버전 업데이트를 배...,O,...,피고 A은 2017. 1. 23. 이 사건 성능조절기능이 포함된 J 버전 업데이트(...,O,2017년 1월 23일에 A가 개발한 이 사건 성능조절기능이 탑재된 J 버전 업데이...,O,피고 A가 개발한 성능조절기능이 포함된 J 버전 업데이트는 2017년 1월 23일에...,O,피고 A가 개발한 성능조절기능이 탑재된 J 버전 업데이트는 2017년 1월 23일에...,O,이 사건 성능조절기능이 탑재된 J 버전 업데이트는 2017년 1월 23일에 배포되었...,O
184,law,이 사건 C의 사용자가 피고 B를 상대로 제기한 손해배상 청구 사건에서는 어떠한 결...,이 사건 C의 사용자가 피고 B를 상대로 서울중앙지방법원에 손해배상을 청구하였지만 ...,[민사] 스마트폰 성능조절기능이 탑재된 소프트웨어 업데이트로 인한 일부 성능제한 또...,0,paragraph,이 사건 C의 사용자가 피고 B를 상대로 제기한 손해배상 청구 사건에서는 제1심에서...,O,이 사건 C의 사용자가 피고 B를 상대로 제기한 손해배상 청구 사건에서는 원고가 패...,O,...,이 사건 C의 다른 사용자가 피고 B를 상대로 서울중앙지방법원 2018가소10083...,O,"서 스마트폰이 정상적으로 사용되지 않았다고 주장했지만, 피고 B를 상대로 한 손해배...",O,이 사건 C의 사용자가 피고 B를 상대로 제기한 손해배상 청구 사건에서는 원고가 패...,O,이 사건 C의 사용자가 피고 B를 상대로 제기한 손해배상 청구 사건에서는 원고의 패...,O,이 사건 C의 사용자가 피고 B를 상대로 제기한 손해배상 청구 사건에서는 원고 패소...,X
185,law,본 문서에서 언급된 피고 A가 고객에게 고지하지 않았던 C의 성능저하 사실은 어떠한...,피고 A는 불완전이행으로 인한 채무불이행책임과 소비자기본법상 고지의무를 위반하였다고...,[민사] 스마트폰 성능조절기능이 탑재된 소프트웨어 업데이트로 인한 일부 성능제한 또...,12,paragraph,본 문서에서 언급된 피고 A가 고객에게 고지하지 않았던 C의 성능저하 사실은 소비자...,O,피고 A가 고객에게 C의 성능저하 사실을 고지하지 않았다는 점은 소비자기본법 제19...,O,...,"본 문서에 따르면, 피고 A가 이 사건 C의 CPU/GPU 등 일부 성능이 제한되고...",O,"피고 A는 고객에게 C의 성능저하 사실을 고지하지 않아 소비자기본법 제19조 제2,...",O,피고 A가 고객에게 고지하지 않았던 C의 성능저하 사실은 계약상 또는 신의칙상 설명...,O,피고 A는 C의 성능저하 사실을 고객에게 고지하지 않았다는 사실로 인해 두 가지 법...,O,본 문서에서 언급된 피고 A가 고객에게 고지하지 않았던 C의 성능저하 사실은 소비자...,O
186,law,소비자기본법에서 규정하는 사업자의 책무는 어떠한 것들이 있나요?,소비자기본법에서 나열하는 '사업자의 책무'에는 물품등으로 인하여 소비자에게 생명·신...,[민사] 스마트폰 성능조절기능이 탑재된 소프트웨어 업데이트로 인한 일부 성능제한 또...,23,paragraph,소비자기본법에서 규정하는 사업자의 책무는 다음과 같습니다. 사업자는 물품이나 용역으...,O,"소비자기본법에 따르면, 사업자는 다음과 같은 책무를 가지고 있습니다. 첫째, 사업자...",O,...,제시된 문맥에는 소비자기본법상 사업자의 책무에 대한 직접적인 언급은 없습니다. 다만...,X,사업자의 책무로는 물품등으로 인한 소비자에게 생명ㆍ신체 또는 재산에 대한 위해가 발...,O,"소비자기본법에 따르면 사업자의 책무는 여러 가지가 있습니다. 첫째, 사업자는 제품이...",O,"소비자기본법에서 규정하는 사업자의 책무는 다음과 같습니다. 첫째, 물품등으로 인해 ...",O,소비자기본법에서 규정하는 사업자의 책무는 다음과 같습니다:\n\n1. 물품등으로 인...,O
187,law,피고 A에게 신의성실의 원칙상 고지의무 혹은 설명의무를 인정해야 하는 이유는 무엇인가?,"피고 A은 이 사건 C의 제조업자이자, 이 사건 C의 운영체제인 I를 제작·배포하는...",[민사] 스마트폰 성능조절기능이 탑재된 소프트웨어 업데이트로 인한 일부 성능제한 또...,28,paragraph,피고 A에게 신의성실의 원칙상 고지의무 혹은 설명의무를 인정해야 하는 이유는 계약 ...,X,피고 A에게 신의성실의 원칙상 고지의무 혹은 설명의무를 인정해야 하는 이유는 부동산...,X,...,피고 A에게 신의성실의 원칙상 고지의무 혹은 설명의무를 인정해야 하는 이유는 다음과...,O,피고 A에게 신의성실의 원칙상 고지의무 혹은 설명의무를 인정해야 하는 이유는 원고들...,X,"피고 A에게 신의성실의 원칙상 고지의무 혹은 설명의무를 인정해야 하는 이유는, 계약...",O,피고 A에게 신의성실의 원칙상 고지의무 혹은 설명의무를 인정해야 하는 이유는 계약의...,O,피고 A에게 신의성실의 원칙상 고지의무 혹은 설명의무를 인정해야 하는 이유는 계약의...,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,commerce,2020년 기업의 자체 라방 거래액이 전체 생방송 라이브커머스 대비 차지하는 비율과...,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,중국21-18호 2021 라이브커머스 산업 보고.pdf,12,paragraph,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,O,2020년에 기업의 자체 라이브 방송 거래액은 전체 생방송 라이브커머스 대비 32....,O,...,2020년 기업의 자체 라이브 방송 거래액이 전체 라이브커머스에서 차지하는 비중은 ...,O,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,O,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,O,2020년에 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 ...,O,2020년 기업의 자체 라방 거래액이 전체 생방송 라이브커머스 대비 차지하는 비율은...,O
296,commerce,"콰이서우와 틱톡에서 거래비율이 급성장한 연도와 그 이유, 그리고 현재 거래액은 어떻...",2020년 콰이서우와 틱톡의 거래비율이 급성장 했습니다. 이는 두 플랫폼이 브랜드 ...,중국21-18호 2021 라이브커머스 산업 보고.pdf,16,paragraph,콰이서우와 틱톡에서 거래비율이 급성장한 연도는 2020년입니다. 이들 플랫폼은 브랜...,O,2020년에 콰이서우와 틱톡 플랫폼 내부 거래비율이 급성장하였습니다. 이는 두 플랫...,O,...,콰이쇼우(Kuaishou)와 틱톡(Douyin)

In [28]:
answers4 = ['Error', 'Error', 'Error']

for index, row in score_df_test.iterrows():
    # 각 행의 파일명과 질문을 사용하여 run_pipeline 실행
    file_path = row['target_file_name']
    question = row['question']
    
    # run_pipeline 함수 호출
    answers4.append(run_pipeline(file_path=file_path, question=question))

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', 

IndexError: list index out of range

In [30]:
len(answers4)

63

In [31]:
my_model_4 = pd.DataFrame(answers4)
my_model_4.columns = ['answer']
my_model_4.to_csv("file/my_model_4.csv")

In [38]:
answer_1 = pd.read_csv("file/my_model.csv")
answer_2 = pd.read_csv("file/my_model_2.csv")
answer_3 = pd.read_csv("file/my_model_3.csv")
answer_4 = pd.read_csv("file/my_model_4.csv")

test_as = pd.concat([answer_1, answer_2, answer_3, answer_4], ignore_index=True)

In [40]:
test_as.drop(columns=['Unnamed: 0'], inplace=True)

In [42]:
test_as

,answer
0,"시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에 차이가 있습니다. 주요 차이..."
1,은행업을 신청할 때 대주주 요건을 충족하기 위해서는 다음 서류들을 제출해야 합니다:...
2,지방은행이 시중은행으로 전환할 때 예비인가를 받을 필요는 없습니다. 이미 인적·물적...
3,은행의 예비인가를 신청하기 위해서는 은행법 제8조제3항 및 제11조를 준수해야 합니...
4,2019년 YTD 기준으로 브라질의 주식 시장 수익률과 베트남의 주식 시장 수익률 ...
...,...
238,1986년에 신설된 공정거래법 제11조의 의결권 제한 제도 입법 취지는 다음과 같습...
239,"2002.1.26 일부개정에 따르면, 의결권을 행사할 수 있는 경우는 다음과 같습니..."
240,코로나 19와 러시아-우크라이나 전쟁의 영향으로 등장한 새로운 소비 트렌드는 이커머...
241,2020년과 2022년에 각각 소비자 체감 물가 및 소비자 동향 지수에 영향을 준 ...


In [56]:
score_df_test = score_df.iloc[249:]
score_df_test

,domain,question,target_answer,target_file_name,target_page_no,context_type,alli_gpt-4-turbo_answer,alli_gpt-4-turbo_ox,alli_gpt-4_answer,alli_gpt-4_ox,...,anything_llm_claude3-opus_answer,anything_llm_claude3-opus_ox,upstage_gpt-3.5-turbo_answer,upstage_gpt-3.5-turbo_ox,upstage_gpt-4-turbo_answer,upstage_gpt-4-turbo_ox,upstage_gpt-4_answer,upstage_gpt-4_ox,upstage_solar_answer,upstage_solar_ox
249,commerce,"애플과 구글이 적용한 개인정보보호 강화 정책으로 인하여, 향후 마케팅 전략에서 어떤...","많은 데이터를 보유한 빅 매체사의 영향력이 더 커질 것이라고 예상되고, 서드파티 쿠...",Dighty Data Insight Report 2023.pdf,4,paragraph,"애플과 구글이 적용한 개인정보보호 강화 정책으로 인해, 향후 마케팅 전략에서는 퍼스...",O,"애플과 구글의 개인정보보호 강화 정책에 따라, 전문가들은 두 가지 주요한 전망을 제...",O,...,"애플과 구글이 개인정보보호 강화 정책을 시행함에 따라, 마케팅 전략에도 변화가 불가...",O,전문가들은 애플과 구글의 개인정보보호 강화 정책으로 인해 퍼스트파티 데이터의 중요성...,O,"애플과 구글의 개인정보보호 강화 정책으로 인해, 마케팅 전략에서 중요해질 점은 두 ...",O,"애플과 구글의 개인정보보호 강화 정책에 따라, 전문가들은 두 가지 주요한 전망을 제...",O,"애플과 구글이 적용한 개인정보보호 강화 정책으로 인하여, 향후 마케팅 전략에서 더욱...",O
250,commerce,재구매 고객의 평균 Historical LTV는 신규구매 고객 대비 얼마나 높은가요?,재구매 고객의 평균 Historical LTV는 신규구매 고객 대비 4.8배 더 높...,Dighty Data Insight Report 2023.pdf,9,paragraph,재구매 고객의 평균 Historical LTV는 신규구매 고객 대비 약 4.8배 높...,O,재구매 고객의 평균 Historical LTV는 신규구매 고객 대비 약 4.8배 더...,O,...,자료에 따르면 재구매 고객의 평균 Historical LTV는 신규구매 고객 대비 ...,O,재구매 고객의 평균 Historical LTV는 신규구매 고객 대비 4.8배 더 높...,O,재구매 고객의 평균 Historical LTV는 신규구매 고객 대비 약 4.8배 높...,O,재구매 고객의 평균 Historical LTV는 신규구매 고객 대비 약 4.8배 높...,O,재구매 고객의 평균 Historical LTV는 신규구매 고객 대비 4.8배 더 높...,O
251,commerce,전체 매출액 중 로얄 고객 10%가 발생시킨 매출액의 비율과 이들의 매출액 비중이 ...,"전체 매출액 중 로얄 고객 10%가 발생시킨 매출액의 비율은 44%이며, 이들의 매...",Dighty Data Insight Report 2023.pdf,11,paragraph,전체 매출액 중 로얄 고객 10%가 발생시킨 매출액의 비율은 44%입니다. 이들의 ...,O,전체 매출액 중 로얄 고객 10%가 발생시킨 매출액의 비율은 44%입니다. 그리고 ...,O,...,자료에 따르면 전체 매출액 중 로열 고객 10%가 발생시킨 매출액 비율은 40%인 ...,X,"로얄 고객 10%가 발생시킨 매출액은 전체 매출액의 44%이며, 특별히 높은 매출액...",O,전체 매출액 중 로얄 고객 10%가 발생시킨 매출액의 비율은 44%입니다. 이들의 ...,O,전체 매출액 중 로얄 고객 10%가 발생시킨 매출액의 비율은 44%입니다. 로얄 고...,O,전체 매출액 중 로얄 고객 10%가 발생시킨 매출액의 비율은 44%입니다. 이들의 ...,O
252,commerce,이커머스 사이트에서 신규 고객 유입보다 재구매 고객의 리텐션을 유지하는 것이 중요하...,신규 고객의 유입만을 생각해서 마케팅을 하다보면 동일한 비용 대비 효과가 점차 하락...,Dighty Data Insight Report 2023.pdf,20,paragraph,"이커머스 사이트에서 재구매 고객의 리텐션을 유지하는 것이 중요한 이유는, 신규 고객...",O,이커머스 사이트에서 신규 고객 유입보다 재구매 고객의 리텐션을 유지하는 것이 중요한...,O,...,이커머스 사이트에서 재구매 고객의 리텐션을 유지하는 것이 신규 고객 유입보다 중요하...,O,"고객 유지 전략은 고객 유입 전략보다 비용이 약 5배 적게 들어가며, 재구매 고객의...",O,"이커머스 사이트에서 재구매 고객의 리텐션을 유지하는 것이 중요한 이유는, 재구매 고...",O,재구매 고객의 리텐션을 유지하는 것이 중요한 이유는 신규 고객 유입에 비해 기존 고...,O,이커머스 사이트에서 신규 고객 유입보다 재구매 고객의 리텐션을 유지하는 것이 중요한...,O
253,commerce,"이커머스 시장 규모가 가장 큰 국가와 그 규모, 이에 비해 가장 작은 규모를 보인 ...","이커머스 시장 규모가 가장 큰 국가는 중국으로, 그 규모는 2,010조원입니다. 이...",쿠팡 상장과 이베이 인수 이후 국내 이커머스 시장은 어떻게 변할까.pdf,1,table,"이커머스 시장 규모가 가장 큰 국가는 중국으로, 전 세계 온라인 소비의 40.5%를...",X,글로벌 이커머스 시장에서 가장 큰 규모를 보이는 국가는 중국입니다. 중국의 온라인 ...,X,...,"제공된 자료에 따르면 이커머스 시장 규모가 가장 큰 국가는 중국이며, 그 규모는 2...",X,"가장 큰 이커머스 시장은 중국으로, 시장 규모는 2,010 조원입니다. 가장 작은 ...",O,"이커머스 시장 규모가 가장 큰 국가는 중국이며, 2020년 기준 시장 규모는 약 2...",O,"이커머스 시장 규모가 가장 큰 국가는 중국으로, 그 규모는 약 2,010조원입니다....",O,"이커머스 시장 규모가 가장 큰 국가는 중국으로, 2020년 시장 규모는 2,010조...",O
254,commerce,2020년 글로벌 이커머스 시장의 규모와 온라인 소비에서 중국이 차지하는 비율은 얼...,"2020년 글로벌 이커머스 시장의 규모는 4.2조 USD(약 5천조원)를 넘어섰고,...",쿠팡 상장과 이베이 인수 이후 국내 이커머스 시장은 어떻게 변할까.pdf,5,paragraph,"2020년 글로벌 이커머스 시장의 규모는 약 4.2조 USD(약 5천조원)이며, 중...",O,2020년 글로벌 이커머스 시장의 규모는 약 4.2조 USD(약 5천조원)였습니다....,O,...,"자료에 따르면 2020년 글로벌 이커머스 시장의 규모는 4조 2,800억 달러로 나...",X,"2020년 중국은 글로벌 이커머스 시장에서 약 40.5%의 점유율을 가지며, 시장규...",O,"2020년 글로벌 이커머스 시장의 규모는 약 5천조원이었으며, 중국은 전 세계 온라...",O,2020년 글로벌 이커머스 시장의 규모는 약 5천조원입니다. 중국의 이커머스 시장 ...,O,2020년 글로벌 이커머스 시장의 규모는 4.2조 USD(약 5천조원)를 넘어섰으며...,O
255,commerce,"이커머스 시장에서 상위 3개 사업자의 점유율이 가장 높은 국가는 어디이며, 그 점유...","상위 3개 사업자의 점유율이 가장 높은 국가는 중국이며, 그 비율은 92.4%, 사...",쿠팡 상장과 이베이 인수 이후 국내 이커머스 시장은 어떻게 변할까.pdf,6,table,"이커머스 시장에서 상위 3개 사업자의 점유율이 가장 높은 국가는 중국이며, 점유율은...",O,"상위 3개 사업자의 점유율이 가장 높은 국가는 중국이며, 그 점유율은 92.4%입니...",X,...,제공된 자료에 따르면 이커머스 시장에서 상위 3개 사업자의 점유율이 가장 높은 국가...,X,"상위 3개 사업자의 점유율이 가장 높은 국가는 중국이며, 상위 사업자는

In [57]:
answers5 = ['Error', 'Error', 'Error', 'Error', 'Error', 'Error', 'Error']

for index, row in score_df_test.iterrows():
    # 각 행의 파일명과 질문을 사용하여 run_pipeline 실행
    file_path = row['target_file_name']
    question = row['question']
    
    # run_pipeline 함수 호출
    answers5.append(run_pipeline(file_path=file_path, question=question))

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', 

In [58]:
my_model_5 = pd.DataFrame(answers5)
my_model_5.columns = ['answer']
my_model_5.to_csv("file/my_model_5.csv")

In [60]:
test_as = pd.concat([test_as, my_model_5], ignore_index=True)

In [70]:
test_as.drop(axis=0, index=243, inplace=True)

In [71]:
test_as

,answer
0,"시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에 차이가 있습니다. 주요 차이..."
1,은행업을 신청할 때 대주주 요건을 충족하기 위해서는 다음 서류들을 제출해야 합니다:...
2,지방은행이 시중은행으로 전환할 때 예비인가를 받을 필요는 없습니다. 이미 인적·물적...
3,은행의 예비인가를 신청하기 위해서는 은행법 제8조제3항 및 제11조를 준수해야 합니...
4,2019년 YTD 기준으로 브라질의 주식 시장 수익률과 베트남의 주식 시장 수익률 ...
...,...
296,2020년 기업의 자체 라방 거래액이 전체 생방송 라이브커머스 대비 차지하는 비율은...
297,"2020년에 콰이서우와 틱톡 플랫폼 내 거래비율이 급성장했으며, 현재 거래액의 약 ..."
298,틱톡과 콰이서우의 서비스 업체 보상 정책은 다음과 같이 서로 다릅니다:\n\n1. ...
299,"한국콘텐츠진흥원 북경비즈니스센터의 상설전시공간에서는 방송, 애니메이션, 캐릭터, 게..."


In [73]:
score_df['My_Model_Answer(Sum_Y,Re_Y)'] = test_as['answer']
score_df

,domain,question,target_answer,target_file_name,target_page_no,context_type,alli_gpt-4-turbo_answer,alli_gpt-4-turbo_ox,alli_gpt-4_answer,alli_gpt-4_ox,...,anything_llm_claude3-opus_ox,upstage_gpt-3.5-turbo_answer,upstage_gpt-3.5-turbo_ox,upstage_gpt-4-turbo_answer,upstage_gpt-4-turbo_ox,upstage_gpt-4_answer,upstage_gpt-4_ox,upstage_solar_answer,upstage_solar_ox,"My_Model_Answer(Sum_Y,Re_Y)"
0,finance,"시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에 차이가 있는데 그 차이점은 ...","시중은행, 지방은행, 인터넷은행 모두 은행업을 영위하기 위해서는 '은행법' 제8조에...",[별첨] 지방은행의 시중은행 전환시 인가방식 및 절차.pdf,4,paragraph,"시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에는 몇 가지 차이점이 있습니다...",O,"시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차는 대부분 동일하지만, 일부 인...",O,...,X,"시중은행, 지방은행, 인터넷은행은 대부분 인가 요건 및 절차가 동일하지만, 최저자본...",O,"시중은행, 지방은행, 인터넷은행 모두 은행법 제8조에 따라 금융위원회의 인가를 받아...",O,"시중은행, 지방은행 및 인터넷은행 모두 은행법 제8조에 따라 금융위원회의 인가를 받...",O,"시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에 차이가 있습니다. 시중은행,...",O,"시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에 차이가 있습니다. 주요 차이..."
1,finance,"은행업을 신청하고자 할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하려면 ...",은행업을 신청하려면 대주주 요건을 충족해야 합니다. 대주주 요건으로는 부실금융기관 ...,[별첨] 지방은행의 시중은행 전환시 인가방식 및 절차.pdf,7,table,"은행업을 신청할 때 은행법상 소유규제에 부합하는 대주주 요건을 충족하기 위해서는 ""...",O,"은행업을 신청하고자 할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하려면 ...",O,...,X,"은행업을 신청하고자 할 때, 대주주 요건을 충족하기 위해 자본금 요건을 충족하기 위...",O,은행업 신청 시 대주주 요건을 충족하기 위해 제출해야 하는 서류로는 주주구성계획이 ...,O,"은행업을 신청하고자 할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하려면 ...",O,"은행업을 신청하고자 할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하려면 ...",O,은행업을 신청할 때 대주주 요건을 충족하기 위해서는 다음 서류들을 제출해야 합니다:...
2,finance,본인가를 받으려는 지방은행이 시중은행 전환시 예비인가를 받을 필요가 있는지 설명하시...,"본인가를 받으려는 지방은행이 시중은행 전환을 신청하는 경우, 예비인가를 받을 필요는...",[별첨] 지방은행의 시중은행 전환시 인가방식 및 절차.pdf,8,paragraph,지방은행이 시중은행으로 전환하려 할 때 예비인가를 받을 필요가 없는 경우가 있습니다...,O,"지방은행이 시중은행으로 전환하려 할 때, 반드시 예비인가를 받을 필요는 없습니다[1...",O,...,X,지방은행이 시중은행 전환을 신청하는 경우 예비인가를 받을 필요는 없다고 판단됩니다....,O,"지방은행이 시중은행으로 전환하고자 할 때, 법적으로 예비인가 절차를 반드시 거치지 ...",O,"지방은행이 시중은행으로 전환하려면 본인가를 받아야하지만, 그 전에 예비인가를 받을 ...",O,"지방은행이 시중은행으로 전환하려는 경우, 예비인가를 받을 필요는 없습니다. 이는 지...",O,지방은행이 시중은행으로 전환할 때 예비인가를 받을 필요는 없습니다. 이미 인적·물적...
3,finance,"은행법에 의거 예비인가를 신청할 수 있는지와, 그 경우 금융위원회가 검토했어야 하는...",은행법에 의하면 예비인가를 신청할 수 있습니다. 제8조에 따른 인가를 받으려는 자는...,[별첨] 지방은행의 시중은행 전환시 인가방식 및 절차.pdf,10,paragraph,"은행법에 따라 예비인가를 신청할 수 있으며, 금융위원회는 예비인가 여부를 결정할 때...",O,"은행법에 따르면, 은행업 인가를 받으려는 자는 미리 금융위원회에 예비인가를 신청할 ...",O,...,X,은행법에 따라 예비인가를 받으려는 자는 미리 금융위원회에 예비인가를 신청할 수 있습...,O,"은행법에 따라 예비인가를 신청할 수 있으며, 이는 제8조에 규정된 본인가를 받기 전...",O,은행법에 따르면 예비인가를 신청할 수 있습니다. 금융위원회는 예비인가를 결정할 때 ...,O,"은행법에 따르면, 예비인가를 신청할 수 있으며, 금융위원회는 예비인가 여부를 결정할...",O,은행의 예비인가를 신청하기 위해서는 은행법 제8조제3항 및 제11조를 준수해야 합니...
4,finance,2019년 YTD 기준으로 브라질의 주식 시장 수익률과 베트남의 주식 시장 수익률 ...,Refinitiv에서 제공한 자료에 따르면 2019년 YTD 브라질의 주식 시장 수...,★2019 제1회 증시콘서트 자료집_최종★.pdf,6,image,2019년 YTD(Year-to-Date) 기준으로 브라질의 주식 시장 수익률에 대...,X,"죄송합니다만, 제공된 정보[1], [2], [3]에서는 2019년 YTD 기준 브라...",X,...,X,2019년 YTD 기준으로 브라질의 주식 시장 수익률과 베트남의 주식 시장 수익률 ...,X,I don't know the answer.,X,"I'm sorry, but the provided context does not c...",X,"2019년 YTD 기준으로 브라질의 주식 시장 수익률은 10.2%이고, 베트남의 주...",X,2019년 YTD 기준으로 브라질의 주식 시장 수익률과 베트남의 주식 시장 수익률 ...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,commerce,2020년 기업의 자체 라방 거래액이 전체 생방송 라이브커머스 대비 차지하는 비율과...,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,중국21-18호 2021 라이브커머스 산업 보고.pdf,12,paragraph,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,O,2020년에 기업의 자체 라이브 방송 거래액은 전체 생방송 라이브커머스 대비 32....,O,...,O,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,O,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,O,2020년에 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 ...,O,2020년 기업의 자체 라방 거래액이 전체 생방송 라이브커머스 대비 차지하는 비율은...,O,"2020년 중국 라이브커머스 시장 규모는 1조 2천억 위안이었으며, 향후 3년간 성..."
296,commerce,"콰이서우와 틱톡에서 거래비율이 급성장한 연도와 그 이유, 그리고 현재 거래액은 어떻...",2020년 콰이서우와 틱톡의 거래비율이 급성장 했습니다. 이는 두 플랫폼이 브랜드 ...,중국21-18호 2021 라이브커머스 산업 보고.pdf,16,paragraph,콰이서우와 틱톡에서 거래비율이 급성장한 연도는 2020년입니다. 이들 플랫폼은 브랜...,O,2020년에 콰이서우와 틱톡 플랫폼 내부 거래비율이 급성장하였습니다. 이는 두 플랫...,O,...,O,"2020년에는 콰이서우와 틱톡의 플랫폼 내부 거래비율이 급성장하여, 현재 거래액의 ...",O,"2020년에 콰이서우와 틱톡 플랫폼 내부의 거래비

In [74]:
drop_ls = []

for i in score_df.columns:
    if "ox" in i:
        drop_ls.append(i)

drop_ls

['alli_gpt-4-turbo_ox',
 'alli_gpt-4_ox',
 'alli_claude3-opus_ox',
 'alli_eeve_ox',
 'alli_Llama-3-Alpha-Ko-8B-Evo_ox',
 'alli_meta-llama3_ox',
 'alli_claude-3.5-sonnet_ox',
 'alli_gpt-4o_ox',
 'alli_meta-llama3_ox.1',
 'langchain_gpt-4-turbo_ox',
 'langchain_gpt-3.5-turbo_ox',
 'openai_assistant_gpt-4-turbo_ox',
 'openai_assistant_gpt-4_ox',
 'cohere_command-r_ox',
 'cohere_command-r-plus_ox',
 'anything_llm_gpt-4-turbo_ox',
 'anything_llm_gpt-3.5-turbo_ox',
 'anything_llm_claude3-opus_ox',
 'upstage_gpt-3.5-turbo_ox',
 'upstage_gpt-4-turbo_ox',
 'upstage_gpt-4_ox',
 'upstage_solar_ox']

In [75]:
score_df.drop(axis=1, columns=drop_ls, inplace=True)

In [76]:
score_df

,domain,question,target_answer,target_file_name,target_page_no,context_type,alli_gpt-4-turbo_answer,alli_gpt-4_answer,alli_claude3-opus_answer,alli_eeve_answer,...,cohere_command-r_answer,cohere_command-r-plus_answer,anything_llm_gpt-4-turbo_answer,anything_llm_gpt-3.5-turbo_answer,anything_llm_claude3-opus_answer,upstage_gpt-3.5-turbo_answer,upstage_gpt-4-turbo_answer,upstage_gpt-4_answer,upstage_solar_answer,"My_Model_Answer(Sum_Y,Re_Y)"
0,finance,"시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에 차이가 있는데 그 차이점은 ...","시중은행, 지방은행, 인터넷은행 모두 은행업을 영위하기 위해서는 '은행법' 제8조에...",[별첨] 지방은행의 시중은행 전환시 인가방식 및 절차.pdf,4,paragraph,"시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에는 몇 가지 차이점이 있습니다...","시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차는 대부분 동일하지만, 일부 인...","시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차의 차이점은 다음과 같습니다. ...","시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에 차이가 있는 이유는 각 은행...",...,"시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차는 대부분 동일하나 일부 요건 ...","시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차는 대부분 동일하지만, 일부 인...","시중은행, 지방은행, 인터넷은행의 인가 요건과 절차는 각각의 은행 종류에 따라 다르...",그 차이점은 주로 관련 법규 및 규제기관에 따라 다를 수 있습니다.,"제공해주신 자료에는 시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차의 차이점에...","시중은행, 지방은행, 인터넷은행은 대부분 인가 요건 및 절차가 동일하지만, 최저자본...","시중은행, 지방은행, 인터넷은행 모두 은행법 제8조에 따라 금융위원회의 인가를 받아...","시중은행, 지방은행 및 인터넷은행 모두 은행법 제8조에 따라 금융위원회의 인가를 받...","시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에 차이가 있습니다. 시중은행,...","시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에 차이가 있습니다. 주요 차이..."
1,finance,"은행업을 신청하고자 할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하려면 ...",은행업을 신청하려면 대주주 요건을 충족해야 합니다. 대주주 요건으로는 부실금융기관 ...,[별첨] 지방은행의 시중은행 전환시 인가방식 및 절차.pdf,7,table,"은행업을 신청할 때 은행법상 소유규제에 부합하는 대주주 요건을 충족하기 위해서는 ""...","은행업을 신청하고자 할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하려면 ...",은행업 인가를 신청할 때 대주주 요건과 관련하여 다음 사항을 충족해야 합니다. [1...,"은행업을 신청하고자 할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하기 위...",...,은행업 대주주로 신청하기 위해선 여러 요건을 충족해야 하는데요. 먼저 자본금이 1천...,"은행업을 신청하고자 할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하기 위...",은행업 신청 시 은행법상 대주주 요건을 충족하기 위해 필요한 서류 제출과 준수해야 ...,"은행법상 대주주 요건을 충족하기 위해선 자산증명서, 소유구조서, 소유주의 개인신상정...",은행법상 은행업 인가를 신청하고자 할 때 대주주 요건과 관련하여 제출해야 할 서류나...,"은행업을 신청하고자 할 때, 대주주 요건을 충족하기 위해 자본금 요건을 충족하기 위...",은행업 신청 시 대주주 요건을 충족하기 위해 제출해야 하는 서류로는 주주구성계획이 ...,"은행업을 신청하고자 할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하려면 ...","은행업을 신청하고자 할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하려면 ...",은행업을 신청할 때 대주주 요건을 충족하기 위해서는 다음 서류들을 제출해야 합니다:...
2,finance,본인가를 받으려는 지방은행이 시중은행 전환시 예비인가를 받을 필요가 있는지 설명하시...,"본인가를 받으려는 지방은행이 시중은행 전환을 신청하는 경우, 예비인가를 받을 필요는...",[별첨] 지방은행의 시중은행 전환시 인가방식 및 절차.pdf,8,paragraph,지방은행이 시중은행으로 전환하려 할 때 예비인가를 받을 필요가 없는 경우가 있습니다...,"지방은행이 시중은행으로 전환하려 할 때, 반드시 예비인가를 받을 필요는 없습니다[1...",지방은행이 시중은행으로 전환할 때 예비인가를 반드시 받을 필요는 없습니다. [1]에...,예비인가를 받지 않고 바로 본인가를 신청하는 지방은행은 이미 인적·물적 설비를 갖추...,...,"지방은행이 시중은행으로 전환하기 위해 본인가를 받으려고 할 때, 반드시 예비인가를 ...","지방은행이 시중은행으로 전환하기 위해 본인가를 신청할 때, 예비인가를 받을 필요는 ...","지방은행이 시중은행으로 전환하고자 하는 경우, 예비인가의 필요성은 다음과 같은 몇 ...",예비인가를 받는 것은 선택사항입니다. 시중은행으로의 전환을 원하는 지방은행이 예비인...,지방은행이 시중은행으로 전환하고자 할 때 예비인가 필요 여부와 관련하여 제공된 자료...,지방은행이 시중은행 전환을 신청하는 경우 예비인가를 받을 필요는 없다고 판단됩니다....,"지방은행이 시중은행으로 전환하고자 할 때, 법적으로 예비인가 절차를 반드시 거치지 ...","지방은행이 시중은행으로 전환하려면 본인가를 받아야하지만, 그 전에 예비인가를 받을 ...","지방은행이 시중은행으로 전환하려는 경우, 예비인가를 받을 필요는 없습니다. 이는 지...",지방은행이 시중은행으로 전환할 때 예비인가를 받을 필요는 없습니다. 이미 인적·물적...
3,finance,"은행법에 의거 예비인가를 신청할 수 있는지와, 그 경우 금융위원회가 검토했어야 하는...",은행법에 의하면 예비인가를 신청할 수 있습니다. 제8조에 따른 인가를 받으려는 자는...,[별첨] 지방은행의 시중은행 전환시 인가방식 및 절차.pdf,10,paragraph,"은행법에 따라 예비인가를 신청할 수 있으며, 금융위원회는 예비인가 여부를 결정할 때...","은행법에 따르면, 은행업 인가를 받으려는 자는 미리 금융위원회에 예비인가를 신청할 ...","네, 은행법 제11조의2에 따르면 은행업 본인가를 받으려는 자는 미리 금융위원회에 ...","은행법에 따르면, 예비인가를 신청할 수 있는 경우는 제8조에 따른 인가를 받으려는 ...",...,은행업 본인가를 받으려는 자는 금융위원회에 예비인가를 신청할 수 있습니다. 지방은행...,은행법에 따라 예비인가를 신청할 수 있습니다. 금융위원회는 예비인가를 받으려는 자가...,은행법에 따라 예비인가는 신규 은행 설립이나 기존 은행의 중대한 변화(예: 지방은행...,예비인가를 신청할 수 있는지 여부는 은행법의 규정에 따라 다를 수 있습니다. 금융위...,은행법상 예비인가 신청 가능 여부와 금융위원회의 검토 요건에 대해서도 제공된 자료에...,은행법에 따라 예비인가를 받으려는 자는 미리 금융위원회에 예비인가를 신청할 수 있습...,"은행법에 따라 예비인가를 신청할 수 있으며, 이는 제8조에 규정된 본인가를 받기 전...",은행법에 따르면 예비인가를 신청할 수 있습니다. 금융위원회는 예비인가를 결정할 때 ...,"은행법에 따르면, 예비인가를 신청할 수 있으며, 금융위원회는 예비인가 여부를 결정할...",은행의 예비인가를 신청하기 위해서는 은행법 제8조제3항 및 제11조를 준수해야 합니...
4,finance,2019년 YTD 기준으로 브라질의 주식 시장 수익률과 베트남의 주식 시장 

In [80]:
score_df.to_csv("file/to_gpt.csv")

In [141]:
fs = pd.read_csv("file/Final_Score(Sun_Y,Re_N).csv")

fs = fs[fs['My Model Answer'] != 'Error']
fs.reset_index(inplace=True)
fs.drop(axis=1, columns=['Unnamed: 0', 'index'], inplace=True)
fs

,domain,question,target_answer,alli_gpt-4-turbo_answer,alli_gpt-4-turbo_answer_cosine_similarity,alli_gpt-4-turbo_answer_binary_Score,alli_gpt-4_answer,alli_gpt-4_answer_cosine_similarity,alli_gpt-4_answer_binary_Score,alli_claude3-opus_answer,...,upstage_gpt-4-turbo_answer_binary_Score,upstage_gpt-4_answer,upstage_gpt-4_answer_cosine_similarity,upstage_gpt-4_answer_binary_Score,upstage_solar_answer,upstage_solar_answer_cosine_similarity,upstage_solar_answer_binary_Score,My Model Answer,My Model Answer_cosine_similarity,My Model Answer_binary_Score
0,finance,"시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에 차이가 있는데 그 차이점은 ...","시중은행, 지방은행, 인터넷은행 모두 은행업을 영위하기 위해서는 '은행법' 제8조에...","시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에는 몇 가지 차이점이 있습니다...",0.605964,O,"시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차는 대부분 동일하지만, 일부 인...",0.656843,O,"시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차의 차이점은 다음과 같습니다. ...",...,O,"시중은행, 지방은행 및 인터넷은행 모두 은행법 제8조에 따라 금융위원회의 인가를 받...",0.495656,O,"시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에 차이가 있습니다. 시중은행,...",0.583009,O,"시중은행, 지방은행, 그리고 인터넷은행의 은행업 인가 요건과 절차에는 몇 가지 주요...",0.379186,O
1,finance,"은행업을 신청하고자 할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하려면 ...",은행업을 신청하려면 대주주 요건을 충족해야 합니다. 대주주 요건으로는 부실금융기관 ...,"은행업을 신청할 때 은행법상 소유규제에 부합하는 대주주 요건을 충족하기 위해서는 ""...",0.771337,O,"은행업을 신청하고자 할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하려면 ...",0.744639,O,은행업 인가를 신청할 때 대주주 요건과 관련하여 다음 사항을 충족해야 합니다. [1...,...,O,"은행업을 신청하고자 할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하려면 ...",0.770738,O,"은행업을 신청하고자 할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하려면 ...",0.787400,O,"은행업을 신청할 때, 대주주 요건을 충족하기 위해서는 다음과 같은 서류들을 제출해야...",0.299868,O
2,finance,본인가를 받으려는 지방은행이 시중은행 전환시 예비인가를 받을 필요가 있는지 설명하시...,"본인가를 받으려는 지방은행이 시중은행 전환을 신청하는 경우, 예비인가를 받을 필요는...",지방은행이 시중은행으로 전환하려 할 때 예비인가를 받을 필요가 없는 경우가 있습니다...,0.650490,O,"지방은행이 시중은행으로 전환하려 할 때, 반드시 예비인가를 받을 필요는 없습니다[1...",0.572233,O,지방은행이 시중은행으로 전환할 때 예비인가를 반드시 받을 필요는 없습니다. [1]에...,...,O,"지방은행이 시중은행으로 전환하려면 본인가를 받아야하지만, 그 전에 예비인가를 받을 ...",0.616264,O,"지방은행이 시중은행으로 전환하려는 경우, 예비인가를 받을 필요는 없습니다. 이는 지...",0.462945,O,본인가를 받으려는 지방은행이 시중은행으로 전환할 때 예비인가를 받을 필요는 없습니다...,0.488536,O
3,finance,"은행법에 의거 예비인가를 신청할 수 있는지와, 그 경우 금융위원회가 검토했어야 하는...",은행법에 의하면 예비인가를 신청할 수 있습니다. 제8조에 따른 인가를 받으려는 자는...,"은행법에 따라 예비인가를 신청할 수 있으며, 금융위원회는 예비인가 여부를 결정할 때...",0.826279,O,"은행법에 따르면, 은행업 인가를 받으려는 자는 미리 금융위원회에 예비인가를 신청할 ...",0.673854,O,"네, 은행법 제11조의2에 따르면 은행업 본인가를 받으려는 자는 미리 금융위원회에 ...",...,O,은행법에 따르면 예비인가를 신청할 수 있습니다. 금융위원회는 예비인가를 결정할 때 ...,0.420982,O,"은행법에 따르면, 예비인가를 신청할 수 있으며, 금융위원회는 예비인가 여부를 결정할...",0.720174,O,은행법 제8조에 따라 은행업 예비인가를 신청할 수 있습니다. 금융위원회가 검토했어야...,0.184690,O
4,finance,2019년 YTD 기준으로 브라질의 주식 시장 수익률과 베트남의 주식 시장 수익률 ...,Refinitiv에서 제공한 자료에 따르면 2019년 YTD 브라질의 주식 시장 수...,2019년 YTD(Year-to-Date) 기준으로 브라질의 주식 시장 수익률에 대...,0.372932,O,"죄송합니다만, 제공된 정보[1], [2], [3]에서는 2019년 YTD 기준 브라...",0.256062,O,"죄송하지만, 제공된 문맥에는 2019년 YTD 기준 브라질과 베트남의 주식시장 수익...",...,O,"I'm sorry, but the provided context does not c...",0.000000,O,"2019년 YTD 기준으로 브라질의 주식 시장 수익률은 10.2%이고, 베트남의 주...",0.473606,O,"2019년 7월 기준으로, 브라질의 주식 시장은 베트남의 주식 시장보다 더 높은 수...",0.227160,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
284,commerce,2020년 기업의 자체 라방 거래액이 전체 생방송 라이브커머스 대비 차지하는 비율과...,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,1.000000,O,2020년에 기업의 자체 라이브 방송 거래액은 전체 생방송 라이브커머스 대비 32....,0.804707,O,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 거래액 대비 32.1...,...,O,2020년에 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 ...,0.883775,O,2020년 기업의 자체 라방 거래액이 전체 생방송 라이브커머스 대비 차지하는 비율은...,0.846829,O,2020년 기업의 자체 라방 거래액이 전체 생방송 라이브커머스 대비 차지하는 비율은...,0.455039,O
285,commerce,"콰이서우와 틱톡에서 거래비율이 급성장한 연도와 그 이유, 그리고 현재 거래액은 어떻...",2020년 콰이서우와 틱톡의 거래비율이 급성장 했습니다. 이는 두 플랫폼이 브랜드 ...,콰이서우와 틱톡에서 거래비율이 급성장한 연도는 2020년입니다. 이들 플랫폼은 브랜...,0.636818,O,2020년에 콰이서우와 틱톡 플랫폼 내부 거래비율이 급성장하였습니다. 이는 두 플랫...,0.618022,O,콰이서우와 틱톡에서 거래비율이 급성장한 시기와 현재 거래액에 대해 다음과 같이 정리...,...,O,2020년에 콰이서우와 틱톡의 거래비율이 급성장했습니다. 이는 다양한 지원 정책 설...,0.430190,O,2020년에 콰이서우와 틱톡의 플랫폼 내부 거래비율이 급성장했습니다. 이는 콰이서우...,0.452579,O,"2020년에 콰이서우와 틱톡 플랫폼 내 거래 비율이 급성장했으며, 현재 거래액의 약...",0.20

In [147]:
drop_ls = []

for i in fs.columns:
    if "answer_cosine_similarity" in i or "answer_binary_Score" in i:
        drop_ls.append(i)

fs.drop(axis=1, columns=drop_ls, inplace=True)
fs.drop(axis=1, columns=['My Model Answer_cosine_similarity', 'My Model Answer_binary_Score'], inplace=True)
fs

,domain,question,target_answer,alli_gpt-4-turbo_answer,alli_gpt-4_answer,alli_claude3-opus_answer,anything_llm_claude3-opus_answer,upstage_gpt-3.5-turbo_answer,upstage_gpt-4-turbo_answer,upstage_gpt-4_answer,upstage_solar_answer,My Model Answer
0,finance,"시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에 차이가 있는데 그 차이점은 ...","시중은행, 지방은행, 인터넷은행 모두 은행업을 영위하기 위해서는 '은행법' 제8조에...","시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에는 몇 가지 차이점이 있습니다...","시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차는 대부분 동일하지만, 일부 인...","시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차의 차이점은 다음과 같습니다. ...","제공해주신 자료에는 시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차의 차이점에...","시중은행, 지방은행, 인터넷은행은 대부분 인가 요건 및 절차가 동일하지만, 최저자본...","시중은행, 지방은행, 인터넷은행 모두 은행법 제8조에 따라 금융위원회의 인가를 받아...","시중은행, 지방은행 및 인터넷은행 모두 은행법 제8조에 따라 금융위원회의 인가를 받...","시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에 차이가 있습니다. 시중은행,...","시중은행, 지방은행, 그리고 인터넷은행의 은행업 인가 요건과 절차에는 몇 가지 주요..."
1,finance,"은행업을 신청하고자 할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하려면 ...",은행업을 신청하려면 대주주 요건을 충족해야 합니다. 대주주 요건으로는 부실금융기관 ...,"은행업을 신청할 때 은행법상 소유규제에 부합하는 대주주 요건을 충족하기 위해서는 ""...","은행업을 신청하고자 할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하려면 ...",은행업 인가를 신청할 때 대주주 요건과 관련하여 다음 사항을 충족해야 합니다. [1...,은행법상 은행업 인가를 신청하고자 할 때 대주주 요건과 관련하여 제출해야 할 서류나...,"은행업을 신청하고자 할 때, 대주주 요건을 충족하기 위해 자본금 요건을 충족하기 위...",은행업 신청 시 대주주 요건을 충족하기 위해 제출해야 하는 서류로는 주주구성계획이 ...,"은행업을 신청하고자 할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하려면 ...","은행업을 신청하고자 할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하려면 ...","은행업을 신청할 때, 대주주 요건을 충족하기 위해서는 다음과 같은 서류들을 제출해야..."
2,finance,본인가를 받으려는 지방은행이 시중은행 전환시 예비인가를 받을 필요가 있는지 설명하시...,"본인가를 받으려는 지방은행이 시중은행 전환을 신청하는 경우, 예비인가를 받을 필요는...",지방은행이 시중은행으로 전환하려 할 때 예비인가를 받을 필요가 없는 경우가 있습니다...,"지방은행이 시중은행으로 전환하려 할 때, 반드시 예비인가를 받을 필요는 없습니다[1...",지방은행이 시중은행으로 전환할 때 예비인가를 반드시 받을 필요는 없습니다. [1]에...,지방은행이 시중은행으로 전환하고자 할 때 예비인가 필요 여부와 관련하여 제공된 자료...,지방은행이 시중은행 전환을 신청하는 경우 예비인가를 받을 필요는 없다고 판단됩니다....,"지방은행이 시중은행으로 전환하고자 할 때, 법적으로 예비인가 절차를 반드시 거치지 ...","지방은행이 시중은행으로 전환하려면 본인가를 받아야하지만, 그 전에 예비인가를 받을 ...","지방은행이 시중은행으로 전환하려는 경우, 예비인가를 받을 필요는 없습니다. 이는 지...",본인가를 받으려는 지방은행이 시중은행으로 전환할 때 예비인가를 받을 필요는 없습니다...
3,finance,"은행법에 의거 예비인가를 신청할 수 있는지와, 그 경우 금융위원회가 검토했어야 하는...",은행법에 의하면 예비인가를 신청할 수 있습니다. 제8조에 따른 인가를 받으려는 자는...,"은행법에 따라 예비인가를 신청할 수 있으며, 금융위원회는 예비인가 여부를 결정할 때...","은행법에 따르면, 은행업 인가를 받으려는 자는 미리 금융위원회에 예비인가를 신청할 ...","네, 은행법 제11조의2에 따르면 은행업 본인가를 받으려는 자는 미리 금융위원회에 ...",은행법상 예비인가 신청 가능 여부와 금융위원회의 검토 요건에 대해서도 제공된 자료에...,은행법에 따라 예비인가를 받으려는 자는 미리 금융위원회에 예비인가를 신청할 수 있습...,"은행법에 따라 예비인가를 신청할 수 있으며, 이는 제8조에 규정된 본인가를 받기 전...",은행법에 따르면 예비인가를 신청할 수 있습니다. 금융위원회는 예비인가를 결정할 때 ...,"은행법에 따르면, 예비인가를 신청할 수 있으며, 금융위원회는 예비인가 여부를 결정할...",은행법 제8조에 따라 은행업 예비인가를 신청할 수 있습니다. 금융위원회가 검토했어야...
4,finance,2019년 YTD 기준으로 브라질의 주식 시장 수익률과 베트남의 주식 시장 수익률 ...,Refinitiv에서 제공한 자료에 따르면 2019년 YTD 브라질의 주식 시장 수...,2019년 YTD(Year-to-Date) 기준으로 브라질의 주식 시장 수익률에 대...,"죄송합니다만, 제공된 정보[1], [2], [3]에서는 2019년 YTD 기준 브라...","죄송하지만, 제공된 문맥에는 2019년 YTD 기준 브라질과 베트남의 주식시장 수익...",제공된 자료에는 2019년 브라질과 베트남의 주식시장 수익률에 대한 직접적인 정보가...,2019년 YTD 기준으로 브라질의 주식 시장 수익률과 베트남의 주식 시장 수익률 ...,I don't know the answer.,"I'm sorry, but the provided context does not c...","2019년 YTD 기준으로 브라질의 주식 시장 수익률은 10.2%이고, 베트남의 주...","2019년 7월 기준으로, 브라질의 주식 시장은 베트남의 주식 시장보다 더 높은 수..."
...,...,...,...,...,...,...,...,...,...,...,...,...
284,commerce,2020년 기업의 자체 라방 거래액이 전체 생방송 라이브커머스 대비 차지하는 비율과...,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,2020년에 기업의 자체 라이브 방송 거래액은 전체 생방송 라이브커머스 대비 32....,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 거래액 대비 32.1...,2020년 기업의 자체 라이브 방송 거래액이 전체 라이브커머스에서 차지하는 비중은 ...,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,2020년에 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 ...,2020년 기업의 자체 라방 거래액이 전체 생방송 라이브커머스 대비 차지하는 비율은...,2020년 기업의 자체 라방 거래액이 전체 생방송 라이브커머스 대비 차지하는 비율은...
285,commerce,"콰이서우와 틱톡에서 거래비율이 급성장한 연도와 그 이유, 그리고 현재 거래액은 어떻...",2020년 콰이서우와 틱톡의 거래비율이 급성장 했습니다. 이는 두 플랫폼이 브랜드 ...,콰이서우와 틱톡에서 거래비율이 급성장한 연도는 2020년입니다. 이들 플랫폼은 브랜...,2020년에 콰이서우와 틱톡 플랫폼 내부 거래비율이 급성장하였습니다. 이는 두 플랫...,콰이서우와 틱톡에서 거래비율이 급성장한 시기와 현재 거래액에 대해 다음과 같이 정리...,콰이쇼우(Kuaishou)와 틱톡(Douyin)은 중국의

In [158]:
to_gpt = pd.read_csv("file/to_gpt.csv")
to_gpt = to_gpt[to_gpt['My_Model_Answer(Sum_Y,Re_Y)'] != 'Error']
to_gpt.reset_index(inplace=True)
to_gpt.drop(axis=1, columns=['index', 'Unnamed: 0', 'target_file_name', 'target_page_no', 'context_type'], inplace=True)
to_gpt

,domain,question,target_answer,alli_gpt-4-turbo_answer,alli_gpt-4_answer,alli_claude3-opus_answer,alli_eeve_answer,alli_Llama-3-Alpha-Ko-8B-Evo_answer,alli_meta-llama3_answer,alli_claude-3.5-sonnet_answer,alli_gpt-4o_answer,alli_meta-llama3_answer.1,langchain_gpt-4-turbo_answer,langchain_gpt-3.5-turbo_answer,openai_assistant_gpt-4-turbo_answer,openai_assistant_gpt-4_answer,cohere_command-r_answer,cohere_command-r-plus_answer,anything_llm_gpt-4-turbo_answer,anything_llm_gpt-3.5-turbo_answer,anything_llm_claude3-opus_answer,upstage_gpt-3.5-turbo_answer,upstage_gpt-4-turbo_answer,upstage_gpt-4_answer,upstage_solar_answer,"My_Model_Answer(Sum_Y,Re_Y)"
0,finance,"시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에 차이가 있는데 그 차이점은 ...","시중은행, 지방은행, 인터넷은행 모두 은행업을 영위하기 위해서는 '은행법' 제8조에...","시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에는 몇 가지 차이점이 있습니다...","시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차는 대부분 동일하지만, 일부 인...","시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차의 차이점은 다음과 같습니다. ...","시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에 차이가 있는 이유는 각 은행...","시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에 차이가 있습니다. 먼저, 시...","According to the provided contexts, the main d...","시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에는 몇 가지 중요한 차이점이 ...","시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에는 몇 가지 차이점이 있습니다...","According to the provided contexts, the main d...","시중은행, 지방은행, 인터넷은행 모두 은행법 제8조에 따라 금융위원회의 인가가 필요...","시중은행, 지방은행, 인터넷은행은 대부분 인가요건 및 절차가 유사하지만 일부 요건 ...","시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에 대한 차이점은 다음과 같습니...","시중은행, 지방은행, 인터넷전문은행의 인가 요건과 절차는 대체로 동일하지만 몇 가지...","시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차는 대부분 동일하나 일부 요건 ...","시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차는 대부분 동일하지만, 일부 인...","시중은행, 지방은행, 인터넷은행의 인가 요건과 절차는 각각의 은행 종류에 따라 다르...",그 차이점은 주로 관련 법규 및 규제기관에 따라 다를 수 있습니다.,"제공해주신 자료에는 시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차의 차이점에...","시중은행, 지방은행, 인터넷은행은 대부분 인가 요건 및 절차가 동일하지만, 최저자본...","시중은행, 지방은행, 인터넷은행 모두 은행법 제8조에 따라 금융위원회의 인가를 받아...","시중은행, 지방은행 및 인터넷은행 모두 은행법 제8조에 따라 금융위원회의 인가를 받...","시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에 차이가 있습니다. 시중은행,...","시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에 차이가 있습니다. 주요 차이..."
1,finance,"은행업을 신청하고자 할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하려면 ...",은행업을 신청하려면 대주주 요건을 충족해야 합니다. 대주주 요건으로는 부실금융기관 ...,"은행업을 신청할 때 은행법상 소유규제에 부합하는 대주주 요건을 충족하기 위해서는 ""...","은행업을 신청하고자 할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하려면 ...",은행업 인가를 신청할 때 대주주 요건과 관련하여 다음 사항을 충족해야 합니다. [1...,"은행업을 신청하고자 할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하기 위...","ID: 1 대주주 요건을 충족하려면 은행법상 소유규제에 부합하는 대주주의 경우, ...","Based on the context [2], to satisfy the big s...",은행업을 신청할 때 대주주 요건을 충족하기 위해 필요한 서류와 지켜야 할 항목에 대...,"은행업을 신청할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하려면 다음과 ...","Based on the context [2], to satisfy the big s...","은행업을 신청할 때, 대주주 요건을 충족하기 위해 제출해야 하는 주요 서류로는 주주...","은행업을 신청하고자 할 때, 자본금 요건을 충족하기 위해 최저 자본금 요건을 충족하...","은행업 신청 시 충족해야 하는 대주주 요건과 관련된 서류 제출에 대해서는, 대주주가...",은행업 인가를 신청하고자 할 때 은행법상 소유규제에 부합하는 대주주 요건을 충족하기...,은행업 대주주로 신청하기 위해선 여러 요건을 충족해야 하는데요. 먼저 자본금이 1천...,"은행업을 신청하고자 할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하기 위...",은행업 신청 시 은행법상 대주주 요건을 충족하기 위해 필요한 서류 제출과 준수해야 ...,"은행법상 대주주 요건을 충족하기 위해선 자산증명서, 소유구조서, 소유주의 개인신상정...",은행법상 은행업 인가를 신청하고자 할 때 대주주 요건과 관련하여 제출해야 할 서류나...,"은행업을 신청하고자 할 때, 대주주 요건을 충족하기 위해 자본금 요건을 충족하기 위...",은행업 신청 시 대주주 요건을 충족하기 위해 제출해야 하는 서류로는 주주구성계획이 ...,"은행업을 신청하고자 할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하려면 ...","은행업을 신청하고자 할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하려면 ...",은행업을 신청할 때 대주주 요건을 충족하기 위해서는 다음 서류들을 제출해야 합니다:...
2,finance,본인가를 받으려는 지방은행이 시중은행 전환시 예비인가를 받을 필요가 있는지 설명하시...,"본인가를 받으려는 지방은행이 시중은행 전환을 신청하는 경우, 예비인가를 받을 필요는...",지방은행이 시중은행으로 전환하려 할 때 예비인가를 받을 필요가 없는 경우가 있습니다...,"지방은행이 시중은행으로 전환하려 할 때, 반드시 예비인가를 받을 필요는 없습니다[1...",지방은행이 시중은행으로 전환할 때 예비인가를 반드시 받을 필요는 없습니다. [1]에...,예비인가를 받지 않고 바로 본인가를 신청하는 지방은행은 이미 인적·물적 설비를 갖추...,ID: 1 본인가를 받으려는 지방은행이 시중은행 전환시 예비인가를 받을 필요가 있...,"Based on the provided context, I'll answer you...",지방은행이 시중은행으로 전환할 때 예비인가를 반드시 받을 필요는 없습니다. 하지만 ...,"지방은행이 시중은행으로 전환할 때 본인가를 받으려는 경우, 예비인가를 반드시 받을 ...","Based on the provided context, I'll answer you...","지방은행이 시중은행으로 전환을 신청하는 경우, 예비인가를 거치지 않아도 됩니다. 이...",지방은행이 이미 은행업을 영위중이고 시중은행 전환을 신청하는 경우에는 예비인가가 필...,"지방은행이 시중은행으로 전환할 때, 예비인가 절차는 생략이 가능합니다. 이는 은행업...","지방은행이 시중은행으로 전환하려는 경우, 은행법 제8조에 

In [159]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

# 모델 로드
model = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

# 함수 정의
def calculate_cosine_similarity(df, target_col, compare_cols):
    # target_answer 열 임베딩
    target_embeddings = model.encode(df[target_col].tolist())
    
    # 결과를 저장할 데이터프레임 복사
    result_df = df.copy()
    
    for col in compare_cols:
        # 비교할 열 임베딩
        compare_embeddings = model.encode(df[col].tolist())
        
        # 코사인 유사도 계산
        similarities = cosine_similarity(target_embeddings, compare_embeddings)
        
        # 유사도 값 저장 (주 대각선 값을 추출)
        result_df[f'{col}_cosine_similarity'] = similarities.diagonal()
    
    return result_df

result = calculate_cosine_similarity(to_gpt, 'target_answer', to_gpt.drop(['target_answer', 'domain', 'question'], axis=1).columns)
result

TypeError: 'float' object is not subscriptable

In [164]:
to_gpt.columns

Index(['domain', 'question', 'target_answer', 'alli_gpt-4-turbo_answer',
       'alli_gpt-4_answer', 'alli_claude3-opus_answer', 'alli_eeve_answer',
       'alli_Llama-3-Alpha-Ko-8B-Evo_answer', 'alli_meta-llama3_answer',
       'alli_claude-3.5-sonnet_answer', 'alli_gpt-4o_answer',
       'alli_meta-llama3_answer.1', 'langchain_gpt-4-turbo_answer',
       'langchain_gpt-3.5-turbo_answer', 'openai_assistant_gpt-4-turbo_answer',
       'openai_assistant_gpt-4_answer', 'cohere_command-r_answer',
       'cohere_command-r-plus_answer', 'anything_llm_gpt-4-turbo_answer',
       'anything_llm_gpt-3.5-turbo_answer', 'anything_llm_claude3-opus_answer',
       'upstage_gpt-3.5-turbo_answer', 'upstage_gpt-4-turbo_answer',
       'upstage_gpt-4_answer', 'upstage_solar_answer',
       'My_Model_Answer(Sum_Y,Re_Y)'],
      dtype='object')

In [165]:
result.columns

Index(['domain', 'question', 'target_answer', 'alli_gpt-4-turbo_answer',
       'alli_gpt-4_answer', 'alli_claude3-opus_answer',
       'anything_llm_claude3-opus_answer', 'upstage_gpt-3.5-turbo_answer',
       'upstage_gpt-4-turbo_answer', 'upstage_gpt-4_answer',
       'upstage_solar_answer', 'My Model Answer',
       'alli_gpt-4-turbo_answer_cosine_similarity',
       'alli_gpt-4_answer_cosine_similarity',
       'alli_claude3-opus_answer_cosine_similarity',
       'anything_llm_claude3-opus_answer_cosine_similarity',
       'upstage_gpt-3.5-turbo_answer_cosine_similarity',
       'upstage_gpt-4-turbo_answer_cosine_similarity',
       'upstage_gpt-4_answer_cosine_similarity',
       'upstage_solar_answer_cosine_similarity',
       'My Model Answer_cosine_similarity'],
      dtype='object')

In [153]:
result.describe().to_csv('file/요약_O_리랭크_X_코사인_유사도.csv')

array([[0.9532292]], dtype=float32)

In [103]:
fs_test

,target_answer,alli_gpt-4-turbo_answer,alli_gpt-4_answer,alli_claude3-opus_answer,anything_llm_claude3-opus_answer,upstage_gpt-3.5-turbo_answer,upstage_gpt-4-turbo_answer,upstage_gpt-4_answer,upstage_solar_answer,My Model Answer
0,"시중은행, 지방은행, 인터넷은행 모두 은행업을 영위하기 위해서는 '은행법' 제8조에...","시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에는 몇 가지 차이점이 있습니다...","시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차는 대부분 동일하지만, 일부 인...","시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차의 차이점은 다음과 같습니다. ...","제공해주신 자료에는 시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차의 차이점에...","시중은행, 지방은행, 인터넷은행은 대부분 인가 요건 및 절차가 동일하지만, 최저자본...","시중은행, 지방은행, 인터넷은행 모두 은행법 제8조에 따라 금융위원회의 인가를 받아...","시중은행, 지방은행 및 인터넷은행 모두 은행법 제8조에 따라 금융위원회의 인가를 받...","시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에 차이가 있습니다. 시중은행,...","시중은행, 지방은행, 그리고 인터넷은행의 은행업 인가 요건과 절차에는 몇 가지 주요..."
1,은행업을 신청하려면 대주주 요건을 충족해야 합니다. 대주주 요건으로는 부실금융기관 ...,"은행업을 신청할 때 은행법상 소유규제에 부합하는 대주주 요건을 충족하기 위해서는 ""...","은행업을 신청하고자 할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하려면 ...",은행업 인가를 신청할 때 대주주 요건과 관련하여 다음 사항을 충족해야 합니다. [1...,은행법상 은행업 인가를 신청하고자 할 때 대주주 요건과 관련하여 제출해야 할 서류나...,"은행업을 신청하고자 할 때, 대주주 요건을 충족하기 위해 자본금 요건을 충족하기 위...",은행업 신청 시 대주주 요건을 충족하기 위해 제출해야 하는 서류로는 주주구성계획이 ...,"은행업을 신청하고자 할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하려면 ...","은행업을 신청하고자 할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하려면 ...","은행업을 신청할 때, 대주주 요건을 충족하기 위해서는 다음과 같은 서류들을 제출해야..."
2,"본인가를 받으려는 지방은행이 시중은행 전환을 신청하는 경우, 예비인가를 받을 필요는...",지방은행이 시중은행으로 전환하려 할 때 예비인가를 받을 필요가 없는 경우가 있습니다...,"지방은행이 시중은행으로 전환하려 할 때, 반드시 예비인가를 받을 필요는 없습니다[1...",지방은행이 시중은행으로 전환할 때 예비인가를 반드시 받을 필요는 없습니다. [1]에...,지방은행이 시중은행으로 전환하고자 할 때 예비인가 필요 여부와 관련하여 제공된 자료...,지방은행이 시중은행 전환을 신청하는 경우 예비인가를 받을 필요는 없다고 판단됩니다....,"지방은행이 시중은행으로 전환하고자 할 때, 법적으로 예비인가 절차를 반드시 거치지 ...","지방은행이 시중은행으로 전환하려면 본인가를 받아야하지만, 그 전에 예비인가를 받을 ...","지방은행이 시중은행으로 전환하려는 경우, 예비인가를 받을 필요는 없습니다. 이는 지...",본인가를 받으려는 지방은행이 시중은행으로 전환할 때 예비인가를 받을 필요는 없습니다...
3,은행법에 의하면 예비인가를 신청할 수 있습니다. 제8조에 따른 인가를 받으려는 자는...,"은행법에 따라 예비인가를 신청할 수 있으며, 금융위원회는 예비인가 여부를 결정할 때...","은행법에 따르면, 은행업 인가를 받으려는 자는 미리 금융위원회에 예비인가를 신청할 ...","네, 은행법 제11조의2에 따르면 은행업 본인가를 받으려는 자는 미리 금융위원회에 ...",은행법상 예비인가 신청 가능 여부와 금융위원회의 검토 요건에 대해서도 제공된 자료에...,은행법에 따라 예비인가를 받으려는 자는 미리 금융위원회에 예비인가를 신청할 수 있습...,"은행법에 따라 예비인가를 신청할 수 있으며, 이는 제8조에 규정된 본인가를 받기 전...",은행법에 따르면 예비인가를 신청할 수 있습니다. 금융위원회는 예비인가를 결정할 때 ...,"은행법에 따르면, 예비인가를 신청할 수 있으며, 금융위원회는 예비인가 여부를 결정할...",은행법 제8조에 따라 은행업 예비인가를 신청할 수 있습니다. 금융위원회가 검토했어야...
4,Refinitiv에서 제공한 자료에 따르면 2019년 YTD 브라질의 주식 시장 수...,2019년 YTD(Year-to-Date) 기준으로 브라질의 주식 시장 수익률에 대...,"죄송합니다만, 제공된 정보[1], [2], [3]에서는 2019년 YTD 기준 브라...","죄송하지만, 제공된 문맥에는 2019년 YTD 기준 브라질과 베트남의 주식시장 수익...",제공된 자료에는 2019년 브라질과 베트남의 주식시장 수익률에 대한 직접적인 정보가...,2019년 YTD 기준으로 브라질의 주식 시장 수익률과 베트남의 주식 시장 수익률 ...,I don't know the answer.,"I'm sorry, but the provided context does not c...","2019년 YTD 기준으로 브라질의 주식 시장 수익률은 10.2%이고, 베트남의 주...","2019년 7월 기준으로, 브라질의 주식 시장은 베트남의 주식 시장보다 더 높은 수..."
...,...,...,...,...,...,...,...,...,...,...
295,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,2020년에 기업의 자체 라이브 방송 거래액은 전체 생방송 라이브커머스 대비 32....,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 거래액 대비 32.1...,2020년 기업의 자체 라이브 방송 거래액이 전체 라이브커머스에서 차지하는 비중은 ...,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,2020년에 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 ...,2020년 기업의 자체 라방 거래액이 전체 생방송 라이브커머스 대비 차지하는 비율은...,2020년 기업의 자체 라방 거래액이 전체 생방송 라이브커머스 대비 차지하는 비율은...
296,2020년 콰이서우와 틱톡의 거래비율이 급성장 했습니다. 이는 두 플랫폼이 브랜드 ...,콰이서우와 틱톡에서 거래비율이 급성장한 연도는 2020년입니다. 이들 플랫폼은 브랜...,2020년에 콰이서우와 틱톡 플랫폼 내부 거래비율이 급성장하였습니다. 이는 두 플랫...,콰이서우와 틱톡에서 거래비율이 급성장한 시기와 현재 거래액에 대해 다음과 같이 정리...,콰이쇼우(Kuaishou)와 틱톡(Douyin)은 중국의 대표적인 숏클립 플랫폼으로...,"2020년에는 콰이서우와 틱톡의 플랫폼 내부 거래비율이 급성장하여, 현재 거래액의 ...","2020년에 콰이서우와 틱톡 플랫폼 내부의 거래비율이 급성장하였으며, 이는 두 플랫...",2020년에 콰이서우와 틱톡의 거래비율이 급성장했습니다. 이는 다양한 지원 정책 설...,2020년에 콰이서우와 틱톡의 플랫폼 내부 거래비율이 급성장했습니다. 이는 콰이서우...,"2020년에 콰이서우와 틱톡 플랫폼 내 거래 비율이 급성장했으며, 현재 거래액의 약..."
297,"콰이서우는 GMV와 서비스 업그레이드를 중시하는 반면, 틱톡은 시장 확대화와 GMV...",틱톡과 콰이서우의 서비스 업체 보상 정책은 다음과 같이 서로 다릅니다. 콰이서...,틱톡과 콰이서우의 서비스 업체 보상 정책은 다음과 같이 서로 다릅니다. 콰이서우는 ...,틱톡과 콰이서우의 서비

In [89]:
fs

,Unnamed: 0,domain,question,target_answer,alli_gpt-4-turbo_answer,alli_gpt-4-turbo_answer_cosine_similarity,alli_gpt-4-turbo_answer_binary_Score,alli_gpt-4_answer,alli_gpt-4_answer_cosine_similarity,alli_gpt-4_answer_binary_Score,...,upstage_gpt-4-turbo_answer_binary_Score,upstage_gpt-4_answer,upstage_gpt-4_answer_cosine_similarity,upstage_gpt-4_answer_binary_Score,upstage_solar_answer,upstage_solar_answer_cosine_similarity,upstage_solar_answer_binary_Score,My Model Answer,My Model Answer_cosine_similarity,My Model Answer_binary_Score
0,0,finance,"시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에 차이가 있는데 그 차이점은 ...","시중은행, 지방은행, 인터넷은행 모두 은행업을 영위하기 위해서는 '은행법' 제8조에...","시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에는 몇 가지 차이점이 있습니다...",0.605964,O,"시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차는 대부분 동일하지만, 일부 인...",0.656843,O,...,O,"시중은행, 지방은행 및 인터넷은행 모두 은행법 제8조에 따라 금융위원회의 인가를 받...",0.495656,O,"시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에 차이가 있습니다. 시중은행,...",0.583009,O,"시중은행, 지방은행, 그리고 인터넷은행의 은행업 인가 요건과 절차에는 몇 가지 주요...",0.379186,O
1,1,finance,"은행업을 신청하고자 할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하려면 ...",은행업을 신청하려면 대주주 요건을 충족해야 합니다. 대주주 요건으로는 부실금융기관 ...,"은행업을 신청할 때 은행법상 소유규제에 부합하는 대주주 요건을 충족하기 위해서는 ""...",0.771337,O,"은행업을 신청하고자 할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하려면 ...",0.744639,O,...,O,"은행업을 신청하고자 할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하려면 ...",0.770738,O,"은행업을 신청하고자 할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하려면 ...",0.787400,O,"은행업을 신청할 때, 대주주 요건을 충족하기 위해서는 다음과 같은 서류들을 제출해야...",0.299868,O
2,2,finance,본인가를 받으려는 지방은행이 시중은행 전환시 예비인가를 받을 필요가 있는지 설명하시...,"본인가를 받으려는 지방은행이 시중은행 전환을 신청하는 경우, 예비인가를 받을 필요는...",지방은행이 시중은행으로 전환하려 할 때 예비인가를 받을 필요가 없는 경우가 있습니다...,0.650490,O,"지방은행이 시중은행으로 전환하려 할 때, 반드시 예비인가를 받을 필요는 없습니다[1...",0.572233,O,...,O,"지방은행이 시중은행으로 전환하려면 본인가를 받아야하지만, 그 전에 예비인가를 받을 ...",0.616264,O,"지방은행이 시중은행으로 전환하려는 경우, 예비인가를 받을 필요는 없습니다. 이는 지...",0.462945,O,본인가를 받으려는 지방은행이 시중은행으로 전환할 때 예비인가를 받을 필요는 없습니다...,0.488536,O
3,3,finance,"은행법에 의거 예비인가를 신청할 수 있는지와, 그 경우 금융위원회가 검토했어야 하는...",은행법에 의하면 예비인가를 신청할 수 있습니다. 제8조에 따른 인가를 받으려는 자는...,"은행법에 따라 예비인가를 신청할 수 있으며, 금융위원회는 예비인가 여부를 결정할 때...",0.826279,O,"은행법에 따르면, 은행업 인가를 받으려는 자는 미리 금융위원회에 예비인가를 신청할 ...",0.673854,O,...,O,은행법에 따르면 예비인가를 신청할 수 있습니다. 금융위원회는 예비인가를 결정할 때 ...,0.420982,O,"은행법에 따르면, 예비인가를 신청할 수 있으며, 금융위원회는 예비인가 여부를 결정할...",0.720174,O,은행법 제8조에 따라 은행업 예비인가를 신청할 수 있습니다. 금융위원회가 검토했어야...,0.184690,O
4,4,finance,2019년 YTD 기준으로 브라질의 주식 시장 수익률과 베트남의 주식 시장 수익률 ...,Refinitiv에서 제공한 자료에 따르면 2019년 YTD 브라질의 주식 시장 수...,2019년 YTD(Year-to-Date) 기준으로 브라질의 주식 시장 수익률에 대...,0.372932,O,"죄송합니다만, 제공된 정보[1], [2], [3]에서는 2019년 YTD 기준 브라...",0.256062,O,...,O,"I'm sorry, but the provided context does not c...",0.000000,O,"2019년 YTD 기준으로 브라질의 주식 시장 수익률은 10.2%이고, 베트남의 주...",0.473606,O,"2019년 7월 기준으로, 브라질의 주식 시장은 베트남의 주식 시장보다 더 높은 수...",0.227160,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,295,commerce,2020년 기업의 자체 라방 거래액이 전체 생방송 라이브커머스 대비 차지하는 비율과...,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,1.000000,O,2020년에 기업의 자체 라이브 방송 거래액은 전체 생방송 라이브커머스 대비 32....,0.804707,O,...,O,2020년에 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 ...,0.883775,O,2020년 기업의 자체 라방 거래액이 전체 생방송 라이브커머스 대비 차지하는 비율은...,0.846829,O,2020년 기업의 자체 라방 거래액이 전체 생방송 라이브커머스 대비 차지하는 비율은...,0.455039,O
296,296,commerce,"콰이서우와 틱톡에서 거래비율이 급성장한 연도와 그 이유, 그리고 현재 거래액은 어떻...",2020년 콰이서우와 틱톡의 거래비율이 급성장 했습니다. 이는 두 플랫폼이 브랜드 ...,콰이서우와 틱톡에서 거래비율이 급성장한 연도는 2020년입니다. 이들 플랫폼은 브랜...,0.636818,O,2020년에 콰이서우와 틱톡 플랫폼 내부 거래비율이 급성장하였습니다. 이는 두 플랫...,0.618022,O,...,O,2020년에 콰이서우와 틱톡의 거래비율이 급성장했습니다. 이는 다양한 지원 정책 설...,0.430190,O,2020년에 콰이서우와 틱톡의 플랫폼 내부 거래비율이 급성장했습니다. 이는 콰이서우...,0.452579,O,"2020년에 콰이서우와 틱톡 플랫폼 내 거래 비율이 급성장했으며, 현재 거래액의 약...",0.200153,O
297,297,commerce,틱톡과 콰이서우의 서비스 업체 보상 정책이 어떻게 서로 다른가요?,"콰이서우는 GMV와 서비스 업그레이드를 중시하는 반면, 틱톡은 시장 확대화와 GMV...",틱톡과 콰이서우의 서비스 업체 보상 정책은 다음과 같이 서로 다릅니다. 콰이서...,0.553389,O,틱톡과 콰이서우의 서비스 업체 보상 정책은 다음과 같이 서로 다릅니다. 콰이서우는 ...,0.544631,O,...,O,콰이서우와 틱톡의 서비스 업체 보상 정책은 서로 다릅니다. 콰이서우는 신뢰에 기반한...,0.309261,O,틱톡과 콰이서우의 서비스 업체 보상 정책의 차이점은 다음과 같습니다.\n\n1. 콰...,0.4

In [88]:
import pandas as pd

fs = pd.read_csv("file/Final_Score(Sun_Y,Re_N).csv")
fs

,Unnamed: 0,domain,question,target_answer,alli_gpt-4-turbo_answer,alli_gpt-4-turbo_answer_cosine_similarity,alli_gpt-4-turbo_answer_binary_Score,alli_gpt-4_answer,alli_gpt-4_answer_cosine_similarity,alli_gpt-4_answer_binary_Score,...,upstage_gpt-4-turbo_answer_binary_Score,upstage_gpt-4_answer,upstage_gpt-4_answer_cosine_similarity,upstage_gpt-4_answer_binary_Score,upstage_solar_answer,upstage_solar_answer_cosine_similarity,upstage_solar_answer_binary_Score,My Model Answer,My Model Answer_cosine_similarity,My Model Answer_binary_Score
0,0,finance,"시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에 차이가 있는데 그 차이점은 ...","시중은행, 지방은행, 인터넷은행 모두 은행업을 영위하기 위해서는 '은행법' 제8조에...","시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에는 몇 가지 차이점이 있습니다...",0.605964,O,"시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차는 대부분 동일하지만, 일부 인...",0.656843,O,...,O,"시중은행, 지방은행 및 인터넷은행 모두 은행법 제8조에 따라 금융위원회의 인가를 받...",0.495656,O,"시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에 차이가 있습니다. 시중은행,...",0.583009,O,"시중은행, 지방은행, 그리고 인터넷은행의 은행업 인가 요건과 절차에는 몇 가지 주요...",0.379186,O
1,1,finance,"은행업을 신청하고자 할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하려면 ...",은행업을 신청하려면 대주주 요건을 충족해야 합니다. 대주주 요건으로는 부실금융기관 ...,"은행업을 신청할 때 은행법상 소유규제에 부합하는 대주주 요건을 충족하기 위해서는 ""...",0.771337,O,"은행업을 신청하고자 할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하려면 ...",0.744639,O,...,O,"은행업을 신청하고자 할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하려면 ...",0.770738,O,"은행업을 신청하고자 할 때, 은행법상 소유규제에 부합하는 대주주 요건을 충족하려면 ...",0.787400,O,"은행업을 신청할 때, 대주주 요건을 충족하기 위해서는 다음과 같은 서류들을 제출해야...",0.299868,O
2,2,finance,본인가를 받으려는 지방은행이 시중은행 전환시 예비인가를 받을 필요가 있는지 설명하시...,"본인가를 받으려는 지방은행이 시중은행 전환을 신청하는 경우, 예비인가를 받을 필요는...",지방은행이 시중은행으로 전환하려 할 때 예비인가를 받을 필요가 없는 경우가 있습니다...,0.650490,O,"지방은행이 시중은행으로 전환하려 할 때, 반드시 예비인가를 받을 필요는 없습니다[1...",0.572233,O,...,O,"지방은행이 시중은행으로 전환하려면 본인가를 받아야하지만, 그 전에 예비인가를 받을 ...",0.616264,O,"지방은행이 시중은행으로 전환하려는 경우, 예비인가를 받을 필요는 없습니다. 이는 지...",0.462945,O,본인가를 받으려는 지방은행이 시중은행으로 전환할 때 예비인가를 받을 필요는 없습니다...,0.488536,O
3,3,finance,"은행법에 의거 예비인가를 신청할 수 있는지와, 그 경우 금융위원회가 검토했어야 하는...",은행법에 의하면 예비인가를 신청할 수 있습니다. 제8조에 따른 인가를 받으려는 자는...,"은행법에 따라 예비인가를 신청할 수 있으며, 금융위원회는 예비인가 여부를 결정할 때...",0.826279,O,"은행법에 따르면, 은행업 인가를 받으려는 자는 미리 금융위원회에 예비인가를 신청할 ...",0.673854,O,...,O,은행법에 따르면 예비인가를 신청할 수 있습니다. 금융위원회는 예비인가를 결정할 때 ...,0.420982,O,"은행법에 따르면, 예비인가를 신청할 수 있으며, 금융위원회는 예비인가 여부를 결정할...",0.720174,O,은행법 제8조에 따라 은행업 예비인가를 신청할 수 있습니다. 금융위원회가 검토했어야...,0.184690,O
4,4,finance,2019년 YTD 기준으로 브라질의 주식 시장 수익률과 베트남의 주식 시장 수익률 ...,Refinitiv에서 제공한 자료에 따르면 2019년 YTD 브라질의 주식 시장 수...,2019년 YTD(Year-to-Date) 기준으로 브라질의 주식 시장 수익률에 대...,0.372932,O,"죄송합니다만, 제공된 정보[1], [2], [3]에서는 2019년 YTD 기준 브라...",0.256062,O,...,O,"I'm sorry, but the provided context does not c...",0.000000,O,"2019년 YTD 기준으로 브라질의 주식 시장 수익률은 10.2%이고, 베트남의 주...",0.473606,O,"2019년 7월 기준으로, 브라질의 주식 시장은 베트남의 주식 시장보다 더 높은 수...",0.227160,O
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,295,commerce,2020년 기업의 자체 라방 거래액이 전체 생방송 라이브커머스 대비 차지하는 비율과...,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,2020년 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 차...,1.000000,O,2020년에 기업의 자체 라이브 방송 거래액은 전체 생방송 라이브커머스 대비 32....,0.804707,O,...,O,2020년에 기업의 자체 라방 거래액은 전체 생방송 라이브커머스 대비 32.1%를 ...,0.883775,O,2020년 기업의 자체 라방 거래액이 전체 생방송 라이브커머스 대비 차지하는 비율은...,0.846829,O,2020년 기업의 자체 라방 거래액이 전체 생방송 라이브커머스 대비 차지하는 비율은...,0.455039,O
296,296,commerce,"콰이서우와 틱톡에서 거래비율이 급성장한 연도와 그 이유, 그리고 현재 거래액은 어떻...",2020년 콰이서우와 틱톡의 거래비율이 급성장 했습니다. 이는 두 플랫폼이 브랜드 ...,콰이서우와 틱톡에서 거래비율이 급성장한 연도는 2020년입니다. 이들 플랫폼은 브랜...,0.636818,O,2020년에 콰이서우와 틱톡 플랫폼 내부 거래비율이 급성장하였습니다. 이는 두 플랫...,0.618022,O,...,O,2020년에 콰이서우와 틱톡의 거래비율이 급성장했습니다. 이는 다양한 지원 정책 설...,0.430190,O,2020년에 콰이서우와 틱톡의 플랫폼 내부 거래비율이 급성장했습니다. 이는 콰이서우...,0.452579,O,"2020년에 콰이서우와 틱톡 플랫폼 내 거래 비율이 급성장했으며, 현재 거래액의 약...",0.200153,O
297,297,commerce,틱톡과 콰이서우의 서비스 업체 보상 정책이 어떻게 서로 다른가요?,"콰이서우는 GMV와 서비스 업그레이드를 중시하는 반면, 틱톡은 시장 확대화와 GMV...",틱톡과 콰이서우의 서비스 업체 보상 정책은 다음과 같이 서로 다릅니다. 콰이서...,0.553389,O,틱톡과 콰이서우의 서비스 업체 보상 정책은 다음과 같이 서로 다릅니다. 콰이서우는 ...,0.544631,O,...,O,콰이서우와 틱톡의 서비스 업체 보상 정책은 서로 다릅니다. 콰이서우는 신뢰에 기반한...,0.309261,O,틱톡과 콰이서우의 서비스 업체 보상 정책의 차이점은 다음과 같습니다.\n\n1. 콰...,0.4

In [2]:
fs.describe()

,Unnamed: 0,alli_gpt-4-turbo_answer_cosine_similarity,alli_gpt-4_answer_cosine_similarity,alli_claude3-opus_answer_cosine_similarity,anything_llm_claude3-opus_answer_cosine_similarity,upstage_gpt-3.5-turbo_answer_cosine_similarity,upstage_gpt-4-turbo_answer_cosine_similarity,upstage_gpt-4_answer_cosine_similarity,upstage_solar_answer_cosine_similarity,My Model Answer_cosine_similarity
count,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000,300.000000
mean,149.500000,0.593712,0.586272,0.506038,0.260912,0.434609,0.468003,0.507805,0.506569,0.360731
std,86.746758,0.220122,0.228378,0.188012,0.146341,0.226020,0.225694,0.247144,0.247006,0.202006
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,74.750000,0.465498,0.428627,0.380567,0.154531,0.285249,0.332919,0.340061,0.332278,0.212276
50%,149.500000,0.629539,0.627914,0.526660,0.251132,0.425369,0.479757,0.536954,0.507054,0.354461
75%,224.250000,0.761999,0.772177,0.654602,0.369312,0.598524,0.621836,0.690417,0.695964,0.488866
max,299.000000,1.000000,1.000000,0.932751,0.670493,1.000000,0.957587,1.000000,1.000000,0.926741


In [4]:
test_df = score_df.iloc[0]

In [6]:
# PyMuPDFLoader 을 이용해 PDF 파일 로드
loader = PyMuPDFLoader(pdf_dir + test_df['target_file_name'])
pages = loader.load()

# KoBART 모델 및 토크나이저 로드
tokenizer = PreTrainedTokenizerFast.from_pretrained("EbanLee/kobart-summary-v3")
bart_model = BartForConditionalGeneration.from_pretrained("EbanLee/kobart-summary-v3")

# RecursiveCharacterTextSplitter를 이용한 텍스트 분할
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=256,
    chunk_overlap=20,
    separators=["\n\n", "\n", "(?<=\. )", " ", ""],
    length_function=lambda text: len(tokenizer.encode(text)),
)

# PDF 문서 분할
docs = text_splitter.split_documents(pages)

# 토크나이징된 문서를 저장할 리스트
tokenized_docs = []
for doc in docs:
    # 각 문서 청크를 토크나이징 (문서를 토큰 리스트로 변환)
    tokenized_text = tokenizer(doc.page_content, truncation=True, padding="max_length", max_length=256, return_tensors="pt")
    tokenized_docs.append(tokenized_text)

# Embeddings 클래스 정의
class MyEmbeddings(Embeddings):
    def __init__(self, model_name="bespin-global/klue-sroberta-base-continue-learning-by-mnr"):
        self.model = SentenceTransformer(model_name)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return self.model.encode(texts, convert_to_tensor=True).tolist()

    def embed_query(self, text: str) -> List[float]:
        return self.embed_documents([text])[0]

RAG_PROMPT_TEMPLATE = (
    "You always answer into Korean. You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. "
    "Use all the sentences and keep the answer concise in bullet.\nQuestion: {question} \nContext: {context} \nAnswer:"
)

SYS_PROMPT_TEMPLATE = (
    "당신은 ** 에이전트입니다. 가능한 한 도움이 될 수 있도록 사실적인 정보만을 제공하도록 설계되었습니다. "
    "친절하게 응대하지만 지나치게 수다스럽지는 않습니다. 아래에 제공된 컨텍스트 정보를 바탕으로, 사전 지식 없이 질문에 답하십시오."
)

# 언어 모델 설정
llm = ChatOpenAI(
    base_url="http://localhost:1234/v1",
    api_key="lm-studio",
    model="teddylee777/EEVE-Korean-Instruct-10.8B-v1.0-gguf",
    temperature=0.1,
)

# 임베딩 모델 설정
emb = MyEmbeddings()

# RAG 체인 설정
rag_prompt = ChatPromptTemplate.from_messages([
    ("system", "You always answer into Korean."),
    ("user", RAG_PROMPT_TEMPLATE),
])
rag_chain = rag_prompt | llm | StrOutputParser()

# 문서 벡터 저장소
vectorstore = FAISS.from_documents(docs, embedding=emb, distance_strategy=DistanceStrategy.COSINE)
retriever = vectorstore.as_retriever(search_kwargs={'k': 5})

# 문서 포맷팅 함수
format_docs = lambda docs: "\n\n".join(doc.page_content for doc in docs)

# 체인 설정
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | rag_chain
)

# KeyBERT 키워드 추출 모델 초기화
keyword_model = KeyBERT()

# 질문 요약 함수
def summarize_question(question: str) -> str:
    # KoBART 토크나이저를 이용해 질문을 토크나이즈
    inputs = tokenizer.encode(question, return_tensors="pt", max_length=1024, truncation=True)
    
    # 요약 생성
    summary_ids = bart_model.generate(inputs, max_length=150, min_length=40, length_penalty=2.0, num_beams=4, early_stopping=True)
    
    # 요약 텍스트 디코딩
    summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
    return summary

# 키워드 추출 함수
def extract_keywords(text: str, top_n: int = 5) -> List[str]:
    keywords = keyword_model.extract_keywords(text, top_n=top_n)
    return [keyword[0] for keyword in keywords]

# 질문을 받으면 요약 및 키워드 추출 후 LLM과 RAG로 전달하는 함수
def process_question(question: str):
    # 질문을 요약
    print(f"질문: {question}")
    
    # 키워드를 추출하여 RAG 시스템에 전달할 준비
    keywords = extract_keywords(question)
    print(f"추출된 키워드: {keywords}")
    
    # RAG 시스템에 키워드를 전달
    retriever_result = retriever.get_relevant_documents(" ".join(keywords))
    
    # 문서에서 컨텍스트를 포맷팅
    context = format_docs(retriever_result)
    
    # RAG 시스템과 LLM에 정보 전달
    input_data = {
        "context": context,
        "question": question
    }
    
    # 결과 생성
    answer = chain.invoke(input_data)
    return answer

# 사용자 질문 예시
question = test_df['question']
answer = process_question(question)
print(f"답변: {answer}")

You passed along `num_labels=3` with an incompatible id to label map: {'0': 'NEGATIVE', '1': 'POSITIVE'}. The number of labels wil be overwritten to 2.
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


질문: 시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에 차이가 있는데 그 차이점은 무엇인가요?
추출된 키워드: ['지방은행', '시중은행', '절차에', '차이점은', '요건']
답변: 시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에 차이가 있습니다. 주요 차이점은 다음과 같습니다:

1. 최저자본금: 시중은행은 1,000억원, 지방은행은 250억원, 인터넷은행도 250억원의 최소 자본금이 필요합니다.
2. 비금융주력자 주식보유한도: 시중은행의 경우 대주주 지분 한도가 4%로 제한되어 있는 반면, 지방은행과 인터넷은행은 각각 15%, 34%까지 허용됩니다.
3. 영업구역 및 방식: 지방은행과 인터넷은행은 전국영업이 아닌 특정 지역 내에서만 영업을 할 수 있으며, 시중은행은 전국에서 영업을 할 수 있습니다.
4. 인가요건 심사: 지방은행의 경우 기존에 은행업 인가를 받았기 때문에 일부 요건만 심사를 받아야 하지만, 시중은행으로 전환할 때는 신규인가와 동일하게 모든 세부심사요건을 심사받아야 합니다.
5. 영업범위 확대: 지방은행이 시중은행으로 전환할 때 종전 대비 영업범위가 확대되기 때문에 사업계획, 내부통제, 임원의 자격요건 등 경영 관련 세부심사요건을 보다 면밀히 심사받게 됩니다.


In [7]:
retriever_result = retriever.get_relevant_documents(" ".join(['지방은행', '시중은행', '절차에', '차이점은', '요건']))

In [8]:
print(score_df.iloc[0]['target_answer'])

시중은행, 지방은행, 인터넷은행 모두 은행업을 영위하기 위해서는 '은행법' 제8조에 근거해 금융위원회의 인가를 필요로 합니다. 그러나 시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에는 일부 차이가 존재합니다.

첫째, 최저자본금의 경우, 시중은행은 1,000억원이 필요하며, 지방은행과 인터넷은행은 250억원이 필요합니다. 

둘째, 비금융주력자 주식 보유한도의 차이가 있습니다. 시중은행의 경우 4%이며, 지방은행은 15%, 인터넷은행은 34%입니다.

셋째, 영업구역 및 영업방식도 차이가 존재합니다. 시중은행과 지방은행은 온라인과 오프라인을 모두 활용하며 전국 또는 일부 지역에서 영업할 수 있지만, 인터넷전문은행은 전국에서 영업하지만 오직 온라인으로만 이루어집니다.


In [9]:
retriever_result

[Document(metadata={'source': 'file/rag_score_pdf/[별첨] 지방은행의 시중은행 전환시 인가방식 및 절차.pdf', 'file_path': 'file/rag_score_pdf/[별첨] 지방은행의 시중은행 전환시 인가방식 및 절차.pdf', 'page': 3, 'total_pages': 15, 'format': 'PDF 1.4', 'title': '', 'author': 'FSC', 'subject': '', 'keywords': '', 'creator': 'Hancom PDF 1.3.0.538', 'producer': 'Hancom PDF 1.3.0.538', 'creationDate': "D:20240131150442+09'00'", 'modDate': "D:20240131150442+09'00'", 'trapped': ''}, page_content='- 2 -\n참고 1  현행 은행업 인가체계\n□ 은행업 영위를 위해서는 「은행법」(제8조)상 금융위 인가필요\n     * (은행법 §8➀) 은행업을 경영하려는 자는 금융위원회의 인가를 받아야 한다.\nㅇ시중은행(전국영업)뿐만 아니라 지방은행 및인터넷은행도 모두 \n동일한 조항(제8조)에 따라 인가 받도록 규정\nㅇ 지방은행 및 인터넷은행은 시중은행과 인가요건·절차가 대부분 \n동일하나, 일부 인가요건* 및 영업구역·방식에서 차이 존재\n     * 최저자본금: (시중銀) 1,000억원 (지방銀) 250억원 (인터넷銀) 250억원\n     * 비금융주력자 주식보유한도: (시중銀) 4% (지방銀) 15% (인터넷銀) 34%\n<표1> 영업구역 및 영업방식에 따른 은행의 종류 구분'),
 Document(metadata={'source': 'file/rag_score_pdf/[별첨] 지방은행의 시중은행 전환시 인가방식 및 절차.pdf', 'file_path': 'file/rag_score_pdf/[별첨] 지방은행의 시중은행 전환시 인가방식 및 절차.pdf', 'page':

In [80]:
retriever_result[0].page_content

'- 2 -\n참고 1  현행 은행업 인가체계\n□ 은행업 영위를 위해서는 「은행법」(제8조)상 금융위 인가필요\n     * (은행법 §8➀) 은행업을 경영하려는 자는 금융위원회의 인가를 받아야 한다.\nㅇ시중은행(전국영업)뿐만 아니라 지방은행 및인터넷은행도 모두 \n동일한 조항(제8조)에 따라 인가 받도록 규정\nㅇ 지방은행 및 인터넷은행은 시중은행과 인가요건·절차가 대부분 \n동일하나, 일부 인가요건* 및 영업구역·방식에서 차이 존재\n     * 최저자본금: (시중銀) 1,000억원 (지방銀) 250억원 (인터넷銀) 250억원\n     * 비금융주력자 주식보유한도: (시중銀) 4% (지방銀) 15% (인터넷銀) 34%\n<표1> 영업구역 및 영업방식에 따른 은행의 종류 구분'

In [81]:
retriever_result[1].page_content

'별첨\n지방은행의 시중은행 전환시\n인가방식 및 절차\n2024. 1.\n금  융  위  원  회\n금  융  감  독  원'

In [19]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np

model_path = "Dongjin-kr/ko-reranker"

def exp_normalize(x):
    b = x.max()
    y = np.exp(x - b)
    return y / y.sum()

from transformers import AutoModelForSequenceClassification, AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()

pairs_test = [
    [test_df['question'], retriever_result[0].page_content],
    [test_df['question'], retriever_result[1].page_content],
    [test_df['question'], retriever_result[2].page_content],
    [test_df['question'], retriever_result[3].page_content],
    [test_df['question'], retriever_result[4].page_content],
]

with torch.no_grad():
    inputs = tokenizer(pairs_test, padding=True, truncation=True, return_tensors='pt', max_length=512)
    scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
    scores = exp_normalize(scores.numpy())
    print(np.round(scores * 100, 2))


[3.088427e+04 6.909326e+04 2.050000e+00 2.017000e+01 2.400000e-01]


In [20]:
for i in scores:
    print(np.round(i * 100, 2))

30.88
69.09
0.0
0.02
0.0


In [30]:
fs.iloc[0]

Unnamed: 0                                                                                            0
domain                                                                                          finance
question                                              시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에 차이가 있는데 그 차이점은 ...
target_answer                                         시중은행, 지방은행, 인터넷은행 모두 은행업을 영위하기 위해서는 '은행법' 제8조에...
alli_gpt-4-turbo_answer                               시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차에는 몇 가지 차이점이 있습니다...
alli_gpt-4-turbo_answer_cosine_similarity                                                      0.605964
alli_gpt-4-turbo_answer_binary_Score                                                                  O
alli_gpt-4_answer                                     시중은행, 지방은행, 인터넷은행의 인가 요건 및 절차는 대부분 동일하지만, 일부 인...
alli_gpt-4_answer_cosine_similarity                                                            0.656843
alli_gpt-4_answer_binary_Score                                  

In [28]:
scores

array([1.8567682e-04, 9.9981433e-01], dtype=float32)

In [33]:
tokenizer = AutoTokenizer.from_pretrained('BAAI/bge-reranker-v2-m3')
model = AutoModelForSequenceClassification.from_pretrained('BAAI/bge-reranker-v2-m3')
model.eval()

with torch.no_grad():
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
    scores = model(**inputs, return_dict=True).logits.view(-1, ).float()
    scores = exp_normalize(scores.numpy()) 
    
print(np.round(scores * 100, 2))

[  0. 100.]


In [16]:
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import numpy as np

model_path = "Dongjin-kr/ko-reranker"

# 모델과 토크나이저 불러오기
tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSequenceClassification.from_pretrained(model_path)
model.eval()

# 여러 문장 비교 예시 (문장 쌍들이 더 많을 경우)
pairs = [
    ["나는 너를 싫어해", "나는 너를 사랑해"], 
    ["나는 너를 좋아해", "너에 대한 나의 감정은 사랑일 수도 있어"], 
    ["나는 너를 존경해", "너에 대한 나의 생각은 긍정적이야"]
]

def rerank(pairs):
    # 입력 데이터 준비
    inputs = tokenizer(pairs, padding=True, truncation=True, return_tensors='pt', max_length=512)
    
    with torch.no_grad():
        # 모델에서 logits 값을 가져옴
        logits = model(**inputs).logits
        # Softmax를 사용하여 확률로 변환
        probs = torch.softmax(logits, dim=-1).view(-1)
        
    # 각 문장의 확률 값 출력
    return probs

# reranking 수행
scores = rerank(pairs)

# 각 문장 쌍에 대한 점수 출력
for i, (pair, score) in enumerate(zip(pairs, scores)):
    print(f"Pair {i + 1}: {pair} => Score: {score.item()}")

Pair 1: ['나는 너를 싫어해', '나는 너를 사랑해'] => Score: 1.0
Pair 2: ['나는 너를 좋아해', '너에 대한 나의 감정은 사랑일 수도 있어'] => Score: 1.0
Pair 3: ['나는 너를 존경해', '너에 대한 나의 생각은 긍정적이야'] => Score: 1.0


In [19]:
fs['domain'].unique()

array(['finance', 'public', 'medical', 'law', 'commerce'], dtype=object)

In [20]:
finance = fs[fs['domain'] == 'finance']
public = fs[fs['domain'] == 'public']
medical = fs[fs['domain'] == 'medical']
law = fs[fs['domain'] == 'law']
commerce = fs[fs['domain'] == 'commerce']

In [21]:
finance.describe()

,Unnamed: 0,alli_gpt-4-turbo_answer_cosine_similarity,alli_gpt-4_answer_cosine_similarity,alli_claude3-opus_answer_cosine_similarity,anything_llm_claude3-opus_answer_cosine_similarity,upstage_gpt-3.5-turbo_answer_cosine_similarity,upstage_gpt-4-turbo_answer_cosine_similarity,upstage_gpt-4_answer_cosine_similarity,upstage_solar_answer_cosine_similarity,My Model Answer_cosine_similarity,alli_gpt-4-turbo_answer_semantic_similarity,My Model Answer_semantic_similarity
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,29.500000,0.565637,0.577046,0.490123,0.244374,0.423051,0.404961,0.450490,0.487590,0.344972,0.670599,0.565356
std,17.464249,0.222615,0.214091,0.190506,0.119574,0.205295,0.235845,0.244747,0.237812,0.171654,0.179539,0.154492
min,0.000000,0.031242,0.010533,0.000000,0.036105,0.017383,0.000000,0.000000,0.000000,0.061958,0.295322,0.291125
25%,14.750000,0.395117,0.416382,0.346569,0.152461,0.296981,0.185002,0.277609,0.339190,0.205918,0.531779,0.443005
50%,29.500000,0.601848,0.613378,0.542958,0.240830,0.424821,0.411315,0.476632,0.479373,0.335104,0.710336,0.545637
75%,44.250000,0.749119,0.722939,0.646030,0.313004,0.543869,0.570060,0.632560,0.642019,0.472261,0.823921,0.663906
max,59.000000,0.895672,0.950191,0.865103,0.495952,0.844048,0.897457,0.884605,0.966823,0.733807,0.972329,0.903171


In [22]:
public.describe()

,Unnamed: 0,alli_gpt-4-turbo_answer_cosine_similarity,alli_gpt-4_answer_cosine_similarity,alli_claude3-opus_answer_cosine_similarity,anything_llm_claude3-opus_answer_cosine_similarity,upstage_gpt-3.5-turbo_answer_cosine_similarity,upstage_gpt-4-turbo_answer_cosine_similarity,upstage_gpt-4_answer_cosine_similarity,upstage_solar_answer_cosine_similarity,My Model Answer_cosine_similarity,alli_gpt-4-turbo_answer_semantic_similarity,My Model Answer_semantic_similarity
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,89.500000,0.584970,0.591781,0.501583,0.199971,0.474975,0.493216,0.538432,0.501883,0.343486,0.640955,0.571866
std,17.464249,0.233666,0.249226,0.219500,0.146002,0.221007,0.215422,0.254137,0.265549,0.176096,0.187295,0.178357
min,60.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.262633,0.246073
25%,74.750000,0.498457,0.488028,0.346774,0.088055,0.349373,0.347494,0.443892,0.326973,0.221082,0.471013,0.403409
50%,89.500000,0.612426,0.634788,0.514402,0.186317,0.486387,0.474171,0.575003,0.522973,0.352391,0.675124,0.572035
75%,104.250000,0.739486,0.781421,0.692490,0.292565,0.611775,0.665262,0.729942,0.711728,0.478730,0.789748,0.705930
max,119.000000,0.963086,0.977400,0.913982,0.649541,0.963086,0.894762,1.000000,1.000000,0.700220,0.978667,0.854020


In [23]:
medical.describe()

,Unnamed: 0,alli_gpt-4-turbo_answer_cosine_similarity,alli_gpt-4_answer_cosine_similarity,alli_claude3-opus_answer_cosine_similarity,anything_llm_claude3-opus_answer_cosine_similarity,upstage_gpt-3.5-turbo_answer_cosine_similarity,upstage_gpt-4-turbo_answer_cosine_similarity,upstage_gpt-4_answer_cosine_similarity,upstage_solar_answer_cosine_similarity,My Model Answer_cosine_similarity,alli_gpt-4-turbo_answer_semantic_similarity,My Model Answer_semantic_similarity
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,149.500000,0.567230,0.562298,0.495660,0.314291,0.450679,0.511218,0.552772,0.504498,0.349737,0.644112,0.614960
std,17.464249,0.217273,0.227568,0.170871,0.130381,0.204830,0.194232,0.222031,0.210167,0.181538,0.169930,0.173142
min,120.000000,0.074206,0.117713,0.175275,0.000000,0.000000,0.047122,0.000000,0.000000,0.000000,0.332515,0.292004
25%,134.750000,0.384862,0.378811,0.380567,0.213052,0.331622,0.393552,0.387889,0.392226,0.235843,0.516899,0.448022
50%,149.500000,0.622729,0.581522,0.476201,0.317130,0.414975,0.490114,0.592134,0.499532,0.340329,0.645346,0.616696
75%,164.250000,0.720520,0.763226,0.648054,0.396606,0.637202,0.626080,0.734387,0.673166,0.455557,0.789722,0.748888
max,179.000000,0.930651,0.919270,0.932751,0.670493,0.794518,0.863946,0.886151,0.896641,0.829514,0.952365,0.905139


In [24]:
law.describe()

,Unnamed: 0,alli_gpt-4-turbo_answer_cosine_similarity,alli_gpt-4_answer_cosine_similarity,alli_claude3-opus_answer_cosine_similarity,anything_llm_claude3-opus_answer_cosine_similarity,upstage_gpt-3.5-turbo_answer_cosine_similarity,upstage_gpt-4-turbo_answer_cosine_similarity,upstage_gpt-4_answer_cosine_similarity,upstage_solar_answer_cosine_similarity,My Model Answer_cosine_similarity,alli_gpt-4-turbo_answer_semantic_similarity,My Model Answer_semantic_similarity
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,209.500000,0.655909,0.637527,0.545420,0.249953,0.383794,0.432992,0.457002,0.486527,0.395047,0.655596,0.607376
std,17.464249,0.190960,0.214751,0.168934,0.174139,0.256836,0.238052,0.269674,0.280632,0.240327,0.177832,0.175344
min,180.000000,0.000000,0.000000,0.158279,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.349635,0.299213
25%,194.750000,0.534706,0.531306,0.462833,0.111415,0.202676,0.253654,0.236494,0.323917,0.201539,0.507825,0.453074
50%,209.500000,0.703031,0.686400,0.571743,0.221195,0.398714,0.473379,0.471638,0.511756,0.415636,0.669245,0.605905
75%,224.250000,0.796990,0.786645,0.687242,0.363601,0.567792,0.622581,0.689133,0.713799,0.593885,0.812668,0.749353
max,239.000000,0.922691,1.000000,0.816110,0.662281,0.922691,0.886380,0.867937,0.957635,0.819575,0.945253,0.944092


In [25]:
commerce.describe()

,Unnamed: 0,alli_gpt-4-turbo_answer_cosine_similarity,alli_gpt-4_answer_cosine_similarity,alli_claude3-opus_answer_cosine_similarity,anything_llm_claude3-opus_answer_cosine_similarity,upstage_gpt-3.5-turbo_answer_cosine_similarity,upstage_gpt-4-turbo_answer_cosine_similarity,upstage_gpt-4_answer_cosine_similarity,upstage_solar_answer_cosine_similarity,My Model Answer_cosine_similarity,alli_gpt-4-turbo_answer_semantic_similarity,My Model Answer_semantic_similarity
count,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000,60.000000
mean,269.500000,0.594812,0.562709,0.497405,0.295972,0.440549,0.497629,0.540330,0.552345,0.370414,0.634441,0.569129
std,17.464249,0.228464,0.233513,0.186788,0.131402,0.235077,0.230199,0.230176,0.237209,0.231602,0.191025,0.176931
min,240.000000,0.000000,0.000000,0.130630,0.000000,0.000000,0.000000,0.000000,0.097830,0.000000,0.297482,0.276654
25%,254.750000,0.481023,0.398672,0.352048,0.205345,0.254904,0.372136,0.392369,0.342118,0.206048,0.501017,0.455749
50%,269.500000,0.623344,0.577443,0.540169,0.278062,0.429197,0.512527,0.534751,0.551847,0.341294,0.620297,0.523947
75%,284.250000,0.749033,0.755754,0.643671,0.392762,0.619130,0.641376,0.730032,0.709999,0.498409,0.778808,0.698791
max,299.000000,1.000000,1.000000,0.852897,0.668224,1.000000,0.957587,0.936736,1.000000,0.926741,0.964978,0.994281


In [40]:
finance[finance['alli_gpt-4-turbo_answer_binary_Score'] == 'O'].count()

Unnamed: 0                                            60
domain                                                60
question                                              60
target_answer                                         60
alli_gpt-4-turbo_answer                               60
alli_gpt-4-turbo_answer_cosine_similarity             60
alli_gpt-4-turbo_answer_binary_Score                  60
alli_gpt-4_answer                                     60
alli_gpt-4_answer_cosine_similarity                   60
alli_gpt-4_answer_binary_Score                        60
alli_claude3-opus_answer                              60
alli_claude3-opus_answer_cosine_similarity            60
alli_claude3-opus_answer_binary_Score                 60
anything_llm_claude3-opus_answer                      60
anything_llm_claude3-opus_answer_cosine_similarity    60
anything_llm_claude3-opus_answer_binary_Score         60
upstage_gpt-3.5-turbo_answer                          60
upstage_gpt-3.5-turbo_answer_co